In [1]:
%%time
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time, psycopg2, os, json
import pandas as pd
from itertools import chain
from sqlalchemy import create_engine
from datetime import datetime, date, timedelta
from collections import Counter

CPU times: total: 1.47 s
Wall time: 2.31 s


---

## Historical Anlysis

In [4]:
from datetime import datetime, date, timedelta
from itertools import chain
from collections import Counter
import time, os, psycopg2, json, requests, re
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

#### League History Extraction

In [5]:
#Dictionary containing the urls of each league for the different sites extracted from.

leagues_list = {
    'English Premier League':['https://www.flashscore.nl/voetbal/engeland/premier-league/schema/', 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/', 'https://www.sofascore.com/tournament/football/england/premier-league/17#52186'],
    'Championship':['https://www.flashscore.nl/voetbal/engeland/championship/schema/','https://www.worldfootball.net/all_matches/eng-championship-2023-2024/'],
    'EFL Championship':['https://www.flashscore.nl/voetbal/engeland/efl-cup/schema/', ''],
    'EFL Trophy':['https://www.flashscore.nl/voetbal/engeland/efl-trophy/schema/', 'https://www.worldfootball.net/all_matches/eng-efl-trophy-2023-2024/'],
    'La Liga':['https://www.flashscore.nl/voetbal/spanje/laliga/schema/','https://www.worldfootball.net/all_matches/esp-primera-division-2023-2024/'],
    'La Liga2':['https://www.flashscore.nl/voetbal/spanje/laliga2/schema/', 'https://www.worldfootball.net/all_matches/esp-segunda-division-2023-2024/'],
    'Bundesliga':['https://www.flashscore.nl/voetbal/duitsland/bundesliga/schema/', 'https://www.worldfootball.net/all_matches/bundesliga-2023-2024/'],
    'Serie A':['https://www.flashscore.nl/voetbal/italie/serie-a/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-a-2023-2024/'],
    'Serie B':['https://www.flashscore.nl/voetbal/italie/serie-b/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-b-2023-2024/'],
    'Ligue 1':['https://www.flashscore.nl/voetbal/frankrijk/ligue-1/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-1-2023-2024/'],
    'Ligue 2':['https://www.flashscore.nl/voetbal/frankrijk/ligue-2/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-2-2023-2024/'],
    'Brasileirão Serie A':['https://www.flashscore.nl/voetbal/brazilie/braziliaanse-competitie/schema/', 'https://www.worldfootball.net/all_matches/bra-serie-a-2023/'],
    'Primera División':['https://www.flashscore.nl/voetbal/argentinie/primera-d/schema/', ''],
    'Major League Soccer':['https://www.flashscore.nl/voetbal/usa/mls/schema/', 'https://www.worldfootball.net/all_matches/usa-major-league-soccer-2023/'],
    'Eredivisie':['https://www.flashscore.nl/eredivisie/schema/', 'https://www.worldfootball.net/all_matches/ned-eredivisie-2023-2024/'],
    'Primeira Liga':['https://www.flashscore.nl/voetbal/portugal/liga-portugal/schema/', 'https://www.worldfootball.net/all_matches/por-primeira-liga-2023-2024/'],
    'J1 League':['https://www.flashscore.nl/voetbal/japan/j1-league/schema/', 'https://www.worldfootball.net/all_matches/jpn-j1-league-2023/'],
    'Scottish Premiership':['https://www.flashscore.nl/voetbal/schotland/premiership/schema/', 'https://www.worldfootball.net/all_matches/sco-premiership-2023-2024/'],
    'Superliga':['https://www.flashscore.nl/voetbal/denemarken/superliga/schema/', 'https://www.worldfootball.net/all_matches/den-superligaen-2023-2024/'],
    'Süper Lig':['https://www.flashscore.nl/voetbal/turkije/super-lig/schema/', 'https://www.worldfootball.net/all_matches/tur-sueperlig-2023-2024/'],
    'Allsvenskan':['https://www.flashscore.nl/voetbal/zweden/allsvenskan/schema/', 'https://www.worldfootball.net/all_matches/swe-allsvenskan-2023/'],
    'Saudi Professional League':['https://www.flashscore.nl/voetbal/saoedi-arabie/premier-league/schema/', 'https://www.worldfootball.net/all_matches/ksa-saudi-pro-league-2023-2024/'],
    'Jupiler Pro League':['https://www.flashscore.nl/voetbal/belgie/jupiler-pro-league/schema/', 'https://www.worldfootball.net/all_matches/bel-eerste-klasse-a-2023-2024/'],
    'UEFA Champions League':['https://www.flashscore.nl/voetbal/europa/champions-league/schema/', 'https://www.worldfootball.net/all_matches/champions-league-2023-2024/'],
    'UEFA Europa League':['https://www.flashscore.nl/voetbal/europa/europa-league/schema/', 'https://www.worldfootball.net/all_matches/europa-league-2023-2024/'],
    'UEFA Europa Conference League':['https://www.flashscore.nl/voetbal/europa/europa-conference-league/schema/', 'https://www.worldfootball.net/all_matches/europa-conference-league-2023-2024/'],
}

In [6]:
def hist_loader(dataframe,table_name):
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
    }

    connection = psycopg2.connect(**db_params)

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

    dataframe.to_sql(table_name, engine, if_exists='append', index=False)

    connection.commit()
    connection.close()
    print('Push to database successfully')

In [7]:
def hist_extraction(leagues_list):
    leagues_dataset = {} #Created the empyt dictionary that will be used to concatenate all table from all leagues

    for key in list(leagues_list.keys()): #Loops through all the leagues in our list of league url
        league_url = leagues_list[key][1]
        leagues_dataset[key] = []
        league_counter = 0
        try:
            if league_url == '': 
                #Checks for empty league links and skips
                continue
            else:
                for year in ['2022-2023','2021-2022','2020-2021']:
                    updated_url = league_url.replace('2023-2024',year)
                    print(updated_url)
                    #Gets the contents from the leagues schedule page
                    response = requests.get(updated_url)
                    time.sleep(2)
                    soup = BeautifulSoup(response.content, "html.parser")

                    #Extracts the table with the match schedules
                    table = soup.find("table", class_="standard_tabelle")

                    data = []
                    match_links = []

                    #Extracts the table rows
                    table_rows = table.find_all("tr")
                    for row in table_rows:
                        #Loops through al the rows and extracts the data in each columns
                        columns = row.find_all("td")
                        row_data = [column.get_text(strip=True) for column in columns]
                        data.append(row_data)

                        #Extracts the urls of each match
                        match_link = row.find("a", href=lambda href: href and "report" in href)
                        if match_link:
                            match_links.append(match_link["href"])
                        else:
                            match_links.append('')

                    #Drops the empty entries
                    zipped_lists = zip(data, match_links)
                    zipped_lists = [pair for pair in zipped_lists if len(pair[0]) > 0]
                    data, match_links = zip(*zipped_lists)

                    columns = ['date', 'time', 'hometeam', 'score', 'awayteam', 'result', 'links']
                    df = pd.DataFrame(data, columns=columns)
                    df.links = match_links

                    df = df[df['links'] != ''] #Drops columns with empty url

                    #Add the prefix to the column
                    df['links'] = 'https://www.worldfootball.net' + df['links']
                    df['date'] = df['date'].replace('', np.nan).ffill()
                    df['date'] = pd.to_datetime(df['date'],  format='%d/%m/%Y')

                    today_df = df.copy(deep=True)
                    curr_league = [key for i in range(len(today_df['links']))]
                    today_df['league'] = curr_league
                    #print(list(today_df['Links']))
                    #print(today_df)
                    leagues_dataset[key].append(today_df)
                    
                for i in range(len(leagues_dataset[key])):
                    if i == 0:
                        final_dataset = leagues_dataset[key][i].copy(deep=True)
                    else:
                        final_dataset = pd.concat([final_dataset, leagues_dataset[key][i]], axis=0) 
                print('Dataset concatenated!')
                
                #Loads the extracted league to the database
                for i in range(2): #Tries twice to load data in case of any unforeseen connection issue
                    try:
                        hist_loader(final_dataset,'leagues_history') #If try is successful, breaks the loop
                        print("All historic matches of {} have been loaded!".format(key))
                        break
                    except Exception as e:
                        except_messgs[str(key)+f": {league_counter} (Database Loading)"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                        if i < 1: #If try isn't successful but it's the first time, then it tries again
                            continue
                        else: #If try isn't successful the second time, it adds the dataframe to the dictionary to try later.
                            leagues_dataset[key] = final_dataset
                        league_counter += 1
  
        except Exception as e:
            print('except')
            except_messgs[str(key)+f": {league_counter}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
            league_counter += 1
            continue
            
        break
    return leagues_dataset

In [77]:
%%time
except_messgs = {}
leagues_dict = hist_extraction(leagues_list)

#### League History Data Wrangling

In [8]:
def hist_reader():
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
        }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    query = f"SELECT * FROM leagues_history"
    cursor.execute(query)

    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cursor.close()
    connection.close()

    # Create a DataFrame from the fetched data
    from_db = pd.DataFrame(result, columns=column_names)
    return from_db

In [5]:
leagues_hist = hist_reader()

In [6]:
leagues_hist.head()

,date,time,hometeam,score,awayteam,result,links,league
0,2022-08-05,20:00,Crystal Palace,-,Arsenal FC,0:2 (0:1),https://www.worldfootball.net/report/premier-l...,English Premier League
1,2022-08-06,12:30,Fulham FC,-,Liverpool FC,2:2 (1:0),https://www.worldfootball.net/report/premier-l...,English Premier League
2,2022-08-06,15:00,AFC Bournemouth,-,Aston Villa,2:0 (1:0),https://www.worldfootball.net/report/premier-l...,English Premier League
3,2022-08-06,15:00,Leeds United,-,Wolverhampton Wanderers,2:1 (1:1),https://www.worldfootball.net/report/premier-l...,English Premier League
4,2022-08-06,15:00,Newcastle United,-,Nottingham Forest,2:0 (0:0),https://www.worldfootball.net/report/premier-l...,English Premier League


In [8]:
leagues_hist.dtypes

date        datetime64[ns]
time                object
hometeam            object
score               object
awayteam            object
result              object
links               object
league              object
dtype: object

In [10]:
leagues_hist = leagues_hist.sort_values(by='date', ascending=False)
leagues_hist.head()

,date,time,hometeam,score,awayteam,result,links,league
15795,2023-06-10,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League
13952,2023-06-07,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig
21231,2023-06-07,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League
13951,2023-06-07,18:00,Beşiktaş,-,Konyaspor,3:3 (2:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig
13950,2023-06-07,18:00,MKE Ankaragücü,-,Adana Demirspor,1:2 (0:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig


In [13]:
#list(leagues_hist.index)

In [14]:
leagues_hist['hometeam'][15795]

'Manchester City'

In [46]:
%%time
home_history = []
away_history = []
head_to_head_history = []
for i in list(leagues_hist.index):
    temp_df = leagues_hist[leagues_hist['date'] < leagues_hist['date'][i]]
    
    #home_history
    temp_df_1 = temp_df[(temp_df['hometeam'] == leagues_hist['hometeam'][i]) | (temp_df['awayteam'] == leagues_hist['hometeam'][i])]
    temp_df_1 = temp_df_1.head(10)
    temp_df_1['date'] = temp_df_1[['date']].astype(str)
    temp_df_1 = temp_df_1.to_dict(orient='list')
    home_history.append(temp_df_1)
    #print(temp_df_1)#.head(6))
    
    #away_history
    temp_df_2 = temp_df[(temp_df['hometeam'] == leagues_hist['awayteam'][i]) | (temp_df['awayteam'] == leagues_hist['awayteam'][i])]
    temp_df_2 = temp_df_2.head(10)
    temp_df_2['date'] = temp_df_2[['date']].astype(str)
    temp_df_2 = temp_df_2.to_dict(orient='list')
    away_history.append(temp_df_2)
    #print(temp_df_2)#.head(6))
    
    #head_to_head_history
    temp_df_3 = temp_df[((temp_df['hometeam'] == leagues_hist['hometeam'][i]) & (temp_df['awayteam'] == leagues_hist['awayteam'][i])) | ((temp_df['awayteam'] == leagues_hist['hometeam'][i]) & (temp_df['hometeam'] == leagues_hist['awayteam'][i]))]
    temp_df_3 = temp_df_3.head(10)
    temp_df_3['date'] = temp_df_3[['date']].astype(str)
    temp_df_3 = temp_df_3.to_dict(orient='list')
    head_to_head_history.append(temp_df_3)
    #print(temp_df_3)#.head(6))

CPU times: total: 7min 59s
Wall time: 8min 6s


In [47]:
print(leagues_hist.shape, len(home_history), len(away_history), len(head_to_head_history))

(21373, 8) 21373 21373 21373


In [48]:
wrangled_history = leagues_hist.copy(deep=True)

In [49]:
wrangled_history['home_history'] = home_history
wrangled_history['away_history'] = away_history
wrangled_history['head_to_head_history'] = head_to_head_history

In [50]:
wrangled_history.head(5)

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history
15795,2023-06-10,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{'date': ['2023-05-28', '2023-05-24', '2023-05...","{'date': ['2023-06-03', '2023-05-27', '2023-05...","{'date': [], 'time': [], 'hometeam': [], 'scor..."
13952,2023-06-07,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-03-20', '2021-11..."
21231,2023-06-07,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{'date': ['2023-06-02', '2023-05-27', '2023-05...","{'date': ['2023-05-28', '2023-05-21', '2023-05...","{'date': [], 'time': [], 'hometeam': [], 'scor..."
13951,2023-06-07,18:00,Beşiktaş,-,Konyaspor,3:3 (2:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-21', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-05-21', '2021-12..."
13950,2023-06-07,18:00,MKE Ankaragücü,-,Adana Demirspor,1:2 (0:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14'], 'time': ['13:00'], 'h..."


In [51]:
wrangled_history['home_history'] = wrangled_history['home_history'].apply(json.dumps)
wrangled_history['away_history'] = wrangled_history['away_history'].apply(json.dumps)
wrangled_history['head_to_head_history'] = wrangled_history['head_to_head_history'].apply(json.dumps)

In [54]:
def transfhist_loader(dataframe):
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
    }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    #Create the table in the database
    create_query = '''CREATE TABLE IF NOT EXISTS leagues_history_transformed (
        date VARCHAR,
        time VARCHAR,
        hometeam VARCHAR,
        score VARCHAR,
        awayteam VARCHAR,
        result VARCHAR,
        links VARCHAR,
        league VARCHAR,
        home_history JSONB,
        away_history JSONB,
        head_to_head_history JSONB
    );'''
    cursor.execute(create_query)
    connection.commit()

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

    dataframe.to_sql('leagues_history_transformed', engine, if_exists='append', index=False)

    connection.commit()
    connection.close()
    print('Push to database successfully')

In [55]:
transfhist_loader(wrangled_history)

Push to database successfully


#### Prediction on Leagues History

In [9]:
def transf_hist_reader():
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
        }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    query = f"SELECT * FROM leagues_history_transformed"
    cursor.execute(query)

    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cursor.close()
    connection.close()

    # Create a DataFrame from the fetched data
    from_db = pd.DataFrame(result, columns=column_names)
    return from_db

In [10]:
def homescore(value):
    '''Function to extract home score from match result'''
    if '(' in value:
        value = (value.replace(' ','').split('('))[0].split(':')[0]
    else:
        value = ''
    return value

def awayscore(value):
    '''Function to extract away score from match result'''
    if '(' in value:
        value = (value.replace(' ','').split('('))[0].split(':')[1]
    else:
        value = ''
    return value   

In [11]:
leagues_abbrv = {
    'English Premier League':'English Premier League','Championship':'Championship',
    'EFL Championship':'EFL Championship','EFL Trophy':'EFL Trophy',
    'La Liga':'La Liga','La Liga2':'La Liga2',
    'Bundesliga':'Bundesliga','Serie A':'Serie A',
    'Serie B':'Serie B','Ligue 1':'Ligue 1',
    'Ligue 2':'Ligue 2','Brasileirão Serie A':'Brasileirão Serie A',
    'Primera División':'Primera División','Major League Soccer':'Major League Soccer',
    'Eredivisie':'Eredivisie','Primeira Liga':'Primeira Liga',
    'J1 League':'J1 League','Scottish Premiership':'Scottish Premiership',
    'Superliga':'Superliga','Süper Lig':'Süper Lig',
    'Allsvenskan':'Allsvenskan','Saudi Professional League':'Saudi Professional League',
    'Jupiler Pro League':'Jupiler Pro League','UEFA Champions League': 'UEFA Champions League',
    'UEFA Europa League':'UEFA Europa League','UEFA Europa Conference League':'UEFA Europa Conference League'
}

In [46]:
def indiv_teamrole_analysis(team_matches, team_name, role, skip=False, by_league=None):
    '''This function filters the table by only matches played by the team under analysis where the said
    team played the role they will play in their upcoming match'''
    #print(team_name, role)
    
    #team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['result'].apply(homescore)
    team_df['away_club_goal'] = team_df['result'].apply(awayscore)
    team_df = team_df[(team_df['home_club_goal'] != '') | (team_df['away_club_goal'] != '')]
    
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
        
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['hometeam']) + list(team_df['awayteam'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} team using {} historic match scores, for only matches where {} team played {} role: '.format(role, role, role, role)
    else:
        end_string = 'Score prediction for both teams using head-to-head historic match scores, for only matches where they played the same role as their upcoming game: '
    
    if by_league != None:
        team_df = team_df[team_df['league'] == by_league[1]]
        
        if num_of_club > 2:
            end_string = 'Score prediction for {} team using {} historic match scores filtered by {}, for only matches where {} team played {} role: '.format(role, role, by_league[0], role, role)
        else:
            end_string = 'Score prediction for both team using head-to-head historic match scores filtered by {}, for only matches where they played the same role as their upcoming game: '.format(by_league[0])
    
    if skip == True:
        limit = team_df.shape[0]
        for i in range(limit):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
                
        if num_of_club > 2:
            end_string = 'Score prediction for {} team using {} historic match scores after skipping rows, for only matches where {} team played {} role: '.format(role, role, role, role)
        else:
            end_string = 'Score prediction for both team using head-to-head historic match scores after skipping rows, for only matches where they played the same role as their upcoming game: '
    
    if role == 'home':
        if len(list(team_df[team_df['hometeam']==team_name])) > 2:
            third_recentscore = [list(team_df[team_df['hometeam']==team_name]['home_club_goal'])[2], list(team_df[team_df['awayteam']!=team_name]['away_club_goal'])[2]]
        else:
            third_recentscore = []
    else:
        if len(list(team_df[team_df['awayteam']==team_name])) > 2:
            third_recentscore = [list(team_df[team_df['hometeam']!=team_name]['home_club_goal'])[2], list(team_df[team_df['awayteam']==team_name]['away_club_goal'])[2]]
        else:
            third_recentscore = []
            
    def check(team_df, column1, column2):
        varb = list((team_df[team_df[column1] == team_name])[column2])[:3] #Filter by a given team in a given role
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
    
    scores = []
    if role == 'home':
        scores.append(check(team_df, 'hometeam', 'home_club_goal'))
        scores.append(check(team_df, 'hometeam', 'away_club_goal'))
    else:
        scores.append(check(team_df, 'awayteam', 'home_club_goal'))
        scores.append(check(team_df, 'awayteam', 'away_club_goal'))
        
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        #Gets all the possible predictions from the observed patterns if any
        for score1 in scores[0]:
            for score2 in scores[1]:
                combined_scores.append([score1, score2])
                
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)            
    else:
        pass
    return final_output


def indiv_role_analysis(team_matches, role, skip=False, by_league=None):
    '''This function analyses the table without any filters and simple checks for pattern for a given
    role (home or away). The historic matches of the team to play the given role in the upcoming match is
    analysed for the given role'''
    #print(role)
    
    #team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['result'].apply(homescore)
    team_df['away_club_goal'] = team_df['result'].apply(awayscore)
    team_df = team_df[(team_df['home_club_goal'] != '') | (team_df['away_club_goal'] != '')]
    
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['hometeam']) + list(team_df['awayteam'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} role using {} historic match scores, regardless of role played by {} team: '.format(role, role, role)
    else:
        end_string = 'Score prediction for both roles using head-to-head historic match scores, regardless of role played by both teams: '
    
    if by_league != None:
        team_df = team_df[team_df['league'] == by_league[1]]
        
        if num_of_club > 2:
            end_string = 'Score prediction for {} role using {} historic match scores filtered by {}, regardless of role played by {} team: '.format(role, role, by_league[0], role)
        else:
            end_string = 'Score prediction for both roles using head-to-head historic match scores filtered by {}, regardless of role played by both teams: '.format(by_league[0])
    
    if skip == True:
        limit = team_df.shape[0]
        for i in range(limit):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
                
        if num_of_club > 2:
            end_string = 'Score prediction for {} role using {} historic match scores after skipping rows, regardless of role played by {} team: '.format(role, role, role)
        else:
            end_string = 'Score prediction for both roles using head-to-head historic match scores after skipping rows, regardless of role played by both teams: '

    
    third_recentscore = [list(team_df['home_club_goal'])[2], list(team_df['away_club_goal'])[2]]
    
    def check(dataframe, column):
        varb = list(dataframe[column])[:3]
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
    
    scores = []    
    scores.append(check(team_df, 'home_club_goal'))
    scores.append(check(team_df, 'away_club_goal'))
    
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        #Gets all the possible predictions from the observed patterns if any
        for score1 in scores[0]:
            for score2 in scores[1]:
                combined_scores.append([score1, score2])
                
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)
    else:
        pass
    return final_output


def indiv_team_analysis(team_matches, team_name, role, skip=False, by_league=None):
    '''This function analyses a given team regardless of which role they played in their historic matches.
    It simply checks how they performed against any opponent and if there are any underlying patterns'''
    #print(team_name, role)
    
    #team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['result'].apply(homescore)
    team_df['away_club_goal'] = team_df['result'].apply(awayscore)
    team_df = team_df[(team_df['home_club_goal'] != '') | (team_df['away_club_goal'] != '')]
    
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['hometeam']) + list(team_df['awayteam'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} team using {} historic match scores, regardless of role played by {} team: '.format(role, role, role)
    else:
        end_string = 'Score prediction for both teams using head-to-head historic match scores, regardless of role played by both teams: '
    
    if by_league != None:
        team_df = team_df[team_df['league'] == by_league[1]]
        
        if num_of_club > 2:
            end_string = 'Score prediction for {} team using {} historic match scores filtered by {}, regardless of role played by {} team: '.format(role, role, by_league[0], role)
        else:
            end_string = 'Score prediction for both teams using head-to-head historic match scores filtered by {}, regardless of role played by both teams: '.format(by_league[0])
    
    if skip == True:
        #print(team_df)
        limit = team_df.shape[0]
        for i in range(limit):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
        
        if num_of_club > 2:
            end_string = 'Score prediction for {} team using {} historic match scores after skipping rows, regardless of role played by {} team: '.format(role, role, role)
        else:
            end_string = 'Score prediction for both teams using head-to-head historic match scores after skipping rows, regardless of role played by both teams: '
        
    
    rearranged = []
    #Rearrangement of the historic table to put the team under analysis in one side and all opponent on the other side.
    for i in range(team_df.shape[0]):
        if list(team_df.iloc[i,:])[2] == team_name:
            rearranged.append(list(team_df.iloc[i,:]))
        else:
            temp_list = list(team_df.iloc[i,:])
            temp_var = temp_list[2]
            temp_list[2] = temp_list[3]
            temp_list[3] = temp_var
            temp_var = temp_list[11]
            temp_list[11] = temp_list[12]
            temp_list[12] = temp_var
            rearranged.append(temp_list)
            
    column = ['date', 'time', 'team', 'score', 'opponent', 'result', 'links',
               'league', 'home_history', 'away_history', 'head_to_head_history',
               'team_score', 'opponent_score']
    new_team_df = pd.DataFrame(rearranged, columns=column)
    
    if role == 'home':
        third_recentscore = [list(new_team_df['team_score'])[2], list(new_team_df['opponent_score'])[2]]
    else:
        third_recentscore = [list(new_team_df['opponent_score'])[2], list(new_team_df['team_score'])[2]]
    
    def check(dataframe, column):
        varb = list(dataframe[column])[:3]
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
            
    scores = []    
    scores.append(check(new_team_df, 'team_score'))
    scores.append(check(new_team_df, 'opponent_score'))
    
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        if role == 'home':
            for score1 in scores[0]:
                for score2 in scores[1]:
                    combined_scores.append([score1, score2])
        else:
            for score1 in scores[1]:
                for score2 in scores[0]:
                    combined_scores.append([score1, score2])
                    
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)
    else:
        pass
    return final_output


def windrawloss_analysis(team_matches, team_name, role, skip=False):
    '''This function analyses the scores of the historic match regardless of team or role and regardless
    of orientation of the scores. Hence if simply gets the match scores and reorders them in descending order
    and check if there's a pattern (repetition). It also check for match outcome pattern (win, loss, draw),
    and if the team under analysis played different roles during this observed pattern'''
    #print(team_name, role)
    
    #team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['result'].apply(homescore)
    team_df['away_club_goal'] = team_df['result'].apply(awayscore)
    team_df = team_df[(team_df['home_club_goal'] != '') | (team_df['away_club_goal'] != '')]
    
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['hometeam']) + list(team_df['awayteam'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} team using {} historic match scores, based on win-loss-draw pattern: '.format(role, role)
    else:
        end_string = 'Score prediction for both teams using head-to-head historic match scores, based on win-loss-draw pattern: '
    
    if skip == True:
        limit = team_df.shape[0]
        for i in range(limit):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
                
        if num_of_club > 2:
            end_string = 'Score prediction for {} team using {} historic match scores after skipping rows, based on win-loss-draw pattern: '.format(role, role)
        else:
            end_string = 'Score prediction for both teams using head-to-head historic match scores after skipping rows, based on win-loss-draw pattern: '
        
    
    overall_cond = [] #The list to compile and check the presence of all three conditions.
    #Check if there is a repeat in score, regardless of team playing a given role or the orientation (home before away)
    scores = []
    for i in range(3):
        temp_scorelist = [list(team_df['home_club_goal'])[i], list(team_df['away_club_goal'])[i]]
        scores.append([max(temp_scorelist), min(temp_scorelist)])
    
    if (scores[0] == scores[1]) | ((scores[0][0] == scores[1][0]) & (abs(scores[0][1] - scores[1][1]) == 1)):
        if scores[2] != scores[0]:
            overall_cond.append('true')
        else:
            overall_cond.append('false')
    else:
        overall_cond.append('false')
        
    #Checks if there roles played by the team were different for the last two games
    roles = [] 
    for i in range(2):
        temp_rolelist = [list(team_df['hometeam'])[i], list(team_df['awayteam'])[i]]
        roles.append(temp_rolelist.index(team_name))
        
    if len(set(roles)) == 2:
        overall_cond.append('true')
    else:
        overall_cond.append('false')
    
    
    #Checks if the outcome (win, loss or draw) was different for the last two games
    rearranged = []
    #Rearrangement
    for i in range(team_df.shape[0]):
        if list(team_df.iloc[i,:])[2] == team_name:
            rearranged.append(list(team_df.iloc[i,:]))
        else:
            temp_list = list(team_df.iloc[i,:])
            temp_var = temp_list[2]
            temp_list[2] = temp_list[3]
            temp_list[3] = temp_var
            temp_var = temp_list[11]
            temp_list[11] = temp_list[12]
            temp_list[12] = temp_var
            rearranged.append(temp_list)
            
    column = ['date', 'time', 'team', 'score', 'opponent', 'result', 'links',
               'league', 'home_history', 'away_history', 'head_to_head_history',
               'team_score', 'opponent_score']
    new_team_df = pd.DataFrame(rearranged, columns=column)
    
    scores = []
    for i in range(3):
        temp_scorelist = [list(new_team_df['team_score'])[i], list(new_team_df['opponent_score'])[i]]
        scores.append(temp_scorelist)
        
    individ_scores = []
    for score in scores[:-1]:
        for individ_score in score:
            individ_scores.append(individ_score)
        
    possible_outcomes = ['win', 'loss', 'draw']
    
    outcomes = []
    for score in scores:
        if score[0] == score[1]:
            outcomes.append('draw')
        elif score[0] > score[1]:
            outcomes.append('win')
        else:
            outcomes.append('loss')
    
    if outcomes[2] in [outcomes[0], outcomes[1]]:
        if outcomes[0] != outcomes[1]:
            overall_cond.append('true')
        else:
            overall_cond.append('false')
    else:
        overall_cond.append('false')
        
    #Compile the final prediction based on whether the three conditions checked for are all present.
    final_output = []
    if 'false' in overall_cond:
        pass
    else:
        prediction = [outcome for outcome in possible_outcomes if outcome not in outcomes]
        if prediction[0] == 'draw':
            elements_counts = Counter(individ_scores)
            final_string = end_string + '{}, {} - {}'.format(prediction[0], elements_counts.most_common()[0][0], elements_counts.most_common()[0][0])
            final_output.append(final_string)
        else:
            elements_counts = Counter(individ_scores)
            final_string = end_string + '{}, {} - {}'.format('win/loss', elements_counts.most_common()[0][0], elements_counts.most_common()[1][0])
            final_output.append(final_string)
    return final_output


def matchscore_total_analysis(dataset, leagues_abbrv):
    '''This function takes in an entire row of the dataset pulled from the database and 
    extracts all the observed pattern from the historic match scores, as well as al the details
    from the inner match details of the historic matches for the home and awar team and their
    head-to-head meatches.'''
    diction = {'8':[2,'home','home_score_patterns'], '9':[4,'away','away_score_patterns'], '10':[2,'home','h2h_score_patterns']}
    columns_list = [8,9,10]
    dict_of_patterns = {'home_score_patterns':[], 'away_score_patterns':[], 'h2h_score_patterns':[]}
    
    for i in range(dataset.shape[0]):
        row = list(dataset.iloc[i,:])
        for number in columns_list:
            dict_of_pattern = {}
            list_of_pattern = []
            
            #This executes all the analysis functions for pattern finding (skip and no skip) to extract predictions
            try:
                patterns = indiv_teamrole_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                #print('\n')
                except_messgs[f"indiv_teamrole_analysis: {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_teamrole_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1], skip=True)
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                #print('\n')
                except_messgs[f"indiv_teamrole_analysis (skip): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_teamrole_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1], by_league=[row[7],leagues_abbrv[row[7]]])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1], [row[7],leagues_abbrv[row[7]]])
                #print('\n')
                except_messgs[f"indiv_teamrole_analysis (by league): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_role_analysis(row[number], diction[str(number)][1])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], diction[str(number)][1])
                #print('\n')
                except_messgs[f"indiv_role_analysis: {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_role_analysis(row[number], diction[str(number)][1], skip=True)
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], diction[str(number)][1])
                #print('\n')
                except_messgs[f"indiv_role_analysis (skip): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_role_analysis(row[number], diction[str(number)][1], by_league=[row[7],leagues_abbrv[row[7]]])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], diction[str(number)][1], [row[7],leagues_abbrv[row[7]]])
                #print('\n')
                except_messgs[f"indiv_role_analysis (by league): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_team_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                #print('\n')
                except_messgs[f"indiv_team_analysis: {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_team_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1], skip=True)
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                #print('\n')
                except_messgs[f"indiv_team_analysis (skip): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = indiv_team_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1], by_league=[row[7],leagues_abbrv[row[7]]])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1], [row[7],leagues_abbrv[row[7]]])
                #print('\n')
                except_messgs[f"indiv_team_analysis (by league): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = windrawloss_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                #print('\n')
                except_messgs[f"windrawloss_analysis: {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []
            try:
                patterns = windrawloss_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1], skip=True)
                list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            except Exception as e:
                #print(row[number], row[diction[str(number)][0]], diction[str(number)][1])
                #print('\n')
                except_messgs[f"windrawloss_analysis (skip): {str(i)}"] = f"{type(e).__name__}: {e}" #Catches and Records Error
                list_of_pattern = list_of_pattern + []

            #Adds all the predictions for a given team history to one list to be alter added to the final dataset
            for i in range(len(list_of_pattern)):
                dict_of_pattern[str(i)] = list_of_pattern[i]

            dict_of_patterns[diction[str(number)][2]].append(json.dumps(dict_of_pattern))
    return dict_of_patterns


def team_analysis_flow(match_df):
    #Team Match Analysis
    try:
        #Converts the date column for filtering
        match_df['date'] = pd.to_datetime(match_df['date'], format="%Y-%m-%d %H:%M:%S") #  format='%d/%m/%Y')
    except Exception as e:
        except_messgs[f"(Data Wrangling)"] = f"{type(e).__name__}: {e}" #Catches and Records Error
    
    try:
        #Analyses the dataset and generates the predictions
        additional_columns = matchscore_total_analysis(match_df, leagues_abbrv)
    except Exception as e:
        except_messgs[f"(Data Analysis)"] = f"{type(e).__name__}: {e}" #Catches and Records Error
    
    try:
        modified_dataset = match_df.copy(deep=True)
        #Adds the prediction to the original dataset
        for key in additional_columns.keys():
            modified_dataset[key] = additional_columns[key]
    except Exception as e:
        except_messgs[f"(Prediction Addition)"] = f"{type(e).__name__}: {e}" #Catches and Records Error
        
    return modified_dataset

In [13]:
%%time
transf_df = transf_hist_reader()

CPU times: total: 3.44 s
Wall time: 43.2 s


In [14]:
transf_df.head()

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history
0,2023-06-10 00:00:00,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{'date': ['2023-05-28', '2023-05-24', '2023-05...","{'date': ['2023-06-03', '2023-05-27', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':..."
1,2023-06-07 00:00:00,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-03-20', '2021-11..."
2,2023-06-07 00:00:00,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{'date': ['2023-06-02', '2023-05-27', '2023-05...","{'date': ['2023-05-28', '2023-05-21', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':..."
3,2023-06-07 00:00:00,18:00,Beşiktaş,-,Konyaspor,3:3 (2:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-21', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-05-21', '2021-12..."
4,2023-06-07 00:00:00,18:00,MKE Ankaragücü,-,Adana Demirspor,1:2 (0:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14'], 'time': ['13:00'], 'l..."


In [15]:
transf_df_1 = transf_df.copy(deep=True)

In [16]:
transf_df_1['home_club_goal'] = transf_df_1['result'].apply(homescore)
transf_df_1['away_club_goal'] = transf_df_1['result'].apply(awayscore)

In [17]:
transf_df_1.shape

(21373, 13)

In [18]:
transf_df_1 = transf_df_1[(transf_df_1['home_club_goal'] != '') | (transf_df_1['away_club_goal'] != '')]

In [19]:
transf_df_1.columns

Index(['date', 'time', 'hometeam', 'score', 'awayteam', 'result', 'links',
       'league', 'home_history', 'away_history', 'head_to_head_history',
       'home_club_goal', 'away_club_goal'],
      dtype='object')

In [20]:
transf_df_1.head(3)

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history,home_club_goal,away_club_goal
0,2023-06-10 00:00:00,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{'date': ['2023-05-28', '2023-05-24', '2023-05...","{'date': ['2023-06-03', '2023-05-27', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,0
1,2023-06-07 00:00:00,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-03-20', '2021-11...",2,0
2,2023-06-07 00:00:00,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{'date': ['2023-06-02', '2023-05-27', '2023-05...","{'date': ['2023-05-28', '2023-05-21', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,2


In [21]:
transf_df_1.shape

(21326, 13)

In [49]:
%%time
except_messgs = {}
test_df = transf_df_1.copy(deep=True)
modified_df = team_analysis_flow(test_df)

CPU times: total: 53min 51s
Wall time: 54min 18s


In [68]:
print(modified_df.shape[0])
modified_df.head(3)

21326


,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history,home_club_goal,away_club_goal,home_score_patterns,away_score_patterns,h2h_score_patterns
0,2023-06-10,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{'date': ['2023-05-28', '2023-05-24', '2023-05...","{'date': ['2023-06-03', '2023-05-27', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,0,"{""0"": ""Score prediction for home team using ho...","{""0"": ""Score prediction for away role using aw...",{}
1,2023-06-07,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-03-20', '2021-11...",2,0,{},"{""0"": ""Score prediction for away team using aw...","{""0"": ""Score prediction for both roles using h..."
2,2023-06-07,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{'date': ['2023-06-02', '2023-05-27', '2023-05...","{'date': ['2023-05-28', '2023-05-21', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,2,"{""0"": ""Score prediction for home team using ho...","{""0"": ""Score prediction for away team using aw...",{}


In [57]:
modified_df_1 = modified_df[(modified_df['home_score_patterns'] != json.dumps({})) | (modified_df['away_score_patterns'] != json.dumps({})) | (modified_df['h2h_score_patterns'] != json.dumps({}))]
print(modified_df_1.shape[0])

20429


In [59]:
modified_df_1 = modified_df_1.copy(deep=True)
col_list = ['home_history','away_history','head_to_head_history']
for column in col_list:
    modified_df_1[column] = modified_df_1[column].apply(json.dumps)

In [65]:
modified_df_1.head(3)

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history,home_club_goal,away_club_goal,home_score_patterns,away_score_patterns,h2h_score_patterns
0,2023-06-10,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{""date"": [""2023-05-28"", ""2023-05-24"", ""2023-05...","{""date"": [""2023-06-03"", ""2023-05-27"", ""2023-05...","{""date"": [], ""time"": [], ""links"": [], ""score"":...",1,0,"{""0"": ""Score prediction for home team using ho...","{""0"": ""Score prediction for away role using aw...",{}
1,2023-06-07,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{""date"": [""2023-06-03"", ""2023-05-30"", ""2023-05...","{""date"": [""2023-06-03"", ""2023-05-30"", ""2023-05...","{""date"": [""2023-01-14"", ""2022-03-20"", ""2021-11...",2,0,{},"{""0"": ""Score prediction for away team using aw...","{""0"": ""Score prediction for both roles using h..."
2,2023-06-07,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{""date"": [""2023-06-02"", ""2023-05-27"", ""2023-05...","{""date"": [""2023-05-28"", ""2023-05-21"", ""2023-05...","{""date"": [], ""time"": [], ""links"": [], ""score"":...",1,2,"{""0"": ""Score prediction for home team using ho...","{""0"": ""Score prediction for away team using aw...",{}


In [63]:
def predhist_loader(dataframe):
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
    }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    #Create the table in the database
    create_query = '''CREATE TABLE IF NOT EXISTS leagues_history_predicted (
        date VARCHAR,
        time VARCHAR,
        hometeam VARCHAR,
        score VARCHAR,
        awayteam VARCHAR,
        result VARCHAR,
        links VARCHAR,
        league VARCHAR,
        home_history JSONB,
        away_history JSONB,
        head_to_head_history JSONB,
        home_club_goal VARCHAR,
        away_club_goal VARCHAR,
        home_score_patterns JSONB,
        away_score_patterns JSONB,
        h2h_score_patterns JSONB
    );'''
    cursor.execute(create_query)
    connection.commit()

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

    dataframe.to_sql('leagues_history_predicted', engine, if_exists='append', index=False)

    connection.commit()
    connection.close()
    print('Push to database successfully')

In [64]:
predhist_loader(modified_df_1)

Push to database successfully


#### Prediction Analysis

In [ ]:
# Extracting the accurate predictions

In [2]:
def hist_pred_reader():
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
        }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    query = f"SELECT * FROM leagues_history_predanalysis"#leagues_history_predicted"
    cursor.execute(query)

    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cursor.close()
    connection.close()

    # Create a DataFrame from the fetched data
    from_db = pd.DataFrame(result, columns=column_names)
    return from_db

In [70]:
league_hist_pred = hist_pred_reader()

In [71]:
league_hist_pred.head()

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history,home_club_goal,away_club_goal,home_score_patterns,away_score_patterns,h2h_score_patterns
0,2023-06-10 00:00:00,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{'date': ['2023-05-28', '2023-05-24', '2023-05...","{'date': ['2023-06-03', '2023-05-27', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,0,{'0': 'Score prediction for home team using ho...,{'0': 'Score prediction for away role using aw...,{}
1,2023-06-07 00:00:00,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-03-20', '2021-11...",2,0,{},{'0': 'Score prediction for away team using aw...,{'0': 'Score prediction for both roles using h...
2,2023-06-07 00:00:00,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{'date': ['2023-06-02', '2023-05-27', '2023-05...","{'date': ['2023-05-28', '2023-05-21', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,2,{'0': 'Score prediction for home team using ho...,{'0': 'Score prediction for away team using aw...,{}
3,2023-06-07 00:00:00,18:00,Beşiktaş,-,Konyaspor,3:3 (2:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-21', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-05-21', '2021-12...",3,3,{'0': 'Score prediction for home role using ho...,{'0': 'Score prediction for away team using aw...,{}
4,2023-06-07 00:00:00,18:00,MKE Ankaragücü,-,Adana Demirspor,1:2 (0:1),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14'], 'time': ['13:00'], 'l...",1,2,{},{'0': 'Score prediction for away team using aw...,{}


In [81]:
%%time

#Check for the accurate prediction and record them
correct_pred_list = []
for i in range(league_hist_pred.shape[0]):
    correct_pred = {str(i):[]}
    home_score = str(list(league_hist_pred['home_club_goal'])[i])
    away_score = str(list(league_hist_pred['away_club_goal'])[i])
    columns = ['home_score_patterns','away_score_patterns','h2h_score_patterns']
    for column in columns:
        #print(column)
        for key in list(list(league_hist_pred[column])[i].keys()):
            full_pred = list(league_hist_pred[column])[i][key][:-5]
            #print(full_pred)
            pred_score = list(league_hist_pred[column])[i][key][-5:]
            #print(pred_score)
            pred = pred_score.replace(' ','').split('-')
            #print(pred)
            if (pred[0] == home_score) & (pred[1] == away_score):
                correct_pred[str(i)].append(full_pred)
    correct_pred_list.append(correct_pred)
    #break    

CPU times: total: 31min 4s
Wall time: 31min 54s


In [82]:
correct_pred_list

[{'0': []},
 {'1': []},
 {'2': ['Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: ']},
 {'3': ['Score prediction for home role using home historic match scores, regardless of role played by home team: ',
   'Score prediction for home role using home historic match scores filtered by Süper Lig, regardless of role played by home team: ']},
 {'4': []},
 {'5': []},
 {'6': []},
 {'7': ['Score prediction for home team using home historic match scores, for only matches where home team played home role: ',
   'Score prediction for home team using home historic match scores filtered by Süper Lig, for only matches where home team played home role: ']},
 {'8': []},
 {'9': []},
 {'10': []},
 {'11': ['Score prediction for home team using home historic match scores, for only matches where home team played home role: ',
   'Score prediction for home team using home historic match scores after skipping rows, for only matches w

In [83]:
len(correct_pred_list)

20429

In [84]:
league_hist_pred.shape[0]

20429

In [85]:
league_hist_pred['accurate_prediction'] = correct_pred_list

In [86]:
league_hist_pred.head(3)

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history,home_club_goal,away_club_goal,home_score_patterns,away_score_patterns,h2h_score_patterns,accurate_prediction
0,2023-06-10 00:00:00,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{'date': ['2023-05-28', '2023-05-24', '2023-05...","{'date': ['2023-06-03', '2023-05-27', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,0,{'0': 'Score prediction for home team using ho...,{'0': 'Score prediction for away role using aw...,{},{'0': []}
1,2023-06-07 00:00:00,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-06-03', '2023-05-30', '2023-05...","{'date': ['2023-01-14', '2022-03-20', '2021-11...",2,0,{},{'0': 'Score prediction for away team using aw...,{'0': 'Score prediction for both roles using h...,{'1': []}
2,2023-06-07 00:00:00,20:00,ACF Fiorentina,-,West Ham United,1:2 (0:0),https://www.worldfootball.net/report/europa-co...,UEFA Europa Conference League,"{'date': ['2023-06-02', '2023-05-27', '2023-05...","{'date': ['2023-05-28', '2023-05-21', '2023-05...","{'date': [], 'time': [], 'links': [], 'score':...",1,2,{'0': 'Score prediction for home team using ho...,{'0': 'Score prediction for away team using aw...,{},{'2': ['Score prediction for away role using a...


In [87]:
league_hist_pred_1 = league_hist_pred.copy(deep=True)
col_list = ['home_history','away_history','head_to_head_history','home_score_patterns','away_score_patterns','h2h_score_patterns','accurate_prediction']
for column in col_list:
    league_hist_pred_1[column] = league_hist_pred_1[column].apply(json.dumps)

In [89]:
league_hist_pred_1.head(2)

,date,time,hometeam,score,awayteam,result,links,league,home_history,away_history,head_to_head_history,home_club_goal,away_club_goal,home_score_patterns,away_score_patterns,h2h_score_patterns,accurate_prediction
0,2023-06-10 00:00:00,20:00,Manchester City,-,Inter,1:0 (0:0),https://www.worldfootball.net/report/champions...,UEFA Champions League,"{""date"": [""2023-05-28"", ""2023-05-24"", ""2023-05...","{""date"": [""2023-06-03"", ""2023-05-27"", ""2023-05...","{""date"": [], ""time"": [], ""links"": [], ""score"":...",1,0,"{""0"": ""Score prediction for home team using ho...","{""0"": ""Score prediction for away role using aw...",{},"{""0"": []}"
1,2023-06-07 00:00:00,18:00,Giresunspor,-,Antalyaspor,2:0 (1:0),https://www.worldfootball.net/report/sueperlig...,Süper Lig,"{""date"": [""2023-06-03"", ""2023-05-30"", ""2023-05...","{""date"": [""2023-06-03"", ""2023-05-30"", ""2023-05...","{""date"": [""2023-01-14"", ""2022-03-20"", ""2021-11...",2,0,{},"{""0"": ""Score prediction for away team using aw...","{""0"": ""Score prediction for both roles using h...","{""1"": []}"


In [90]:
def predanalysis_hist_loader(dataframe):
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
    }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    #Create the table in the database
    create_query = '''CREATE TABLE IF NOT EXISTS leagues_history_predanalysis (
        date VARCHAR,
        time VARCHAR,
        hometeam VARCHAR,
        score VARCHAR,
        awayteam VARCHAR,
        result VARCHAR,
        links VARCHAR,
        league VARCHAR,
        home_history JSONB,
        away_history JSONB,
        head_to_head_history JSONB,
        home_club_goal VARCHAR,
        away_club_goal VARCHAR,
        home_score_patterns JSONB,
        away_score_patterns JSONB,
        h2h_score_patterns JSONB,
        accurate_prediction JSONB
    );'''
    cursor.execute(create_query)
    connection.commit()

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

    dataframe.to_sql('leagues_history_predanalysis', engine, if_exists='append', index=False)

    connection.commit()
    connection.close()
    print('Push to database successfully')

In [91]:
predanalysis_hist_loader(league_hist_pred_1)

Push to database successfully


In [ ]:
#Analysing all matches and accurate predictions, overall, overall and by league, season, season and by league
#List of Analysis to be conducted.
#Total accurate prediction -Done
    #By league
    #By Season
    #Overall
    #By season and league
#Count of pattern in accurate prediction -Done
    #By league
    #By Season
    #Overall
    #By season and league
#Top 5 prediction types -Done
#Most frequent teams in accurate predicitons -Done
#Check if there is a trend in the accuracy of the predictions by Weeks interval -Done

In [3]:
%%time
corr_pred = hist_pred_reader()

CPU times: total: 2.14 s
Wall time: 41.5 s


In [4]:
%%time
df_corr_pred = corr_pred[['date','hometeam','awayteam','league','home_club_goal','away_club_goal','accurate_prediction']]
df_corr_pred = df_corr_pred.copy(deep=True)
df_corr_pred.head(3)

CPU times: total: 15.6 ms
Wall time: 16 ms


,date,hometeam,awayteam,league,home_club_goal,away_club_goal,accurate_prediction
0,2023-06-10 00:00:00,Manchester City,Inter,UEFA Champions League,1,0,"{""0"": []}"
1,2023-06-07 00:00:00,Giresunspor,Antalyaspor,Süper Lig,2,0,"{""1"": []}"
2,2023-06-07 00:00:00,ACF Fiorentina,West Ham United,UEFA Europa Conference League,1,2,"{""2"": [""Score prediction for away role using a..."


In [5]:
%%time
df_corr_pred['accurate_prediction'] = df_corr_pred['accurate_prediction'].apply(json.loads)

CPU times: total: 109 ms
Wall time: 80.7 ms


In [6]:
%%time
df_corr_pred['date'] = pd.to_datetime(df_corr_pred['date'], format="%Y-%m-%d %H:%M:%S")

CPU times: total: 15.6 ms
Wall time: 12.2 ms


In [7]:
df_corr_pred.head(3)

,date,hometeam,awayteam,league,home_club_goal,away_club_goal,accurate_prediction
0,2023-06-10,Manchester City,Inter,UEFA Champions League,1,0,{'0': []}
1,2023-06-07,Giresunspor,Antalyaspor,Süper Lig,2,0,{'1': []}
2,2023-06-07,ACF Fiorentina,West Ham United,UEFA Europa Conference League,1,2,{'2': ['Score prediction for away role using a...


In [8]:
leagues = [
    'English Premier League','Championship',
    'EFL Championship','EFL Trophy',
    'La Liga','La Liga2',
    'Bundesliga','Serie A',
    'Serie B','Ligue 1',
    'Ligue 2','Brasileirão Serie A',
    'Primera División','Major League Soccer',
    'Eredivisie','Primeira Liga',
    'J1 League','Scottish Premiership',
    'Superliga','Süper Lig',
    'Allsvenskan','Saudi Professional League',
    'Jupiler Pro League','UEFA Champions League',
    'UEFA Europa League','UEFA Europa Conference League'
    ]

seasons = {
    'English Premier League':[('08/05/2023','05/08/2022'),('22/05/2022','13/08/2021'),('23/05/2021','12/09/2020')],
    'Championship':[('08/05/2023','29/07/2022'),('07/05/2022','06/08/2021'),('08/05/2021','11/09/2020')],
    'EFL Championship':[],
    'EFL Trophy':[('02/04/2023','30/08/2022'),('03/04/2022','31/08/2021'),('14/03/2021','01/09/2020')],
    'La Liga':[('04/06/2023','12/08/2022'),('22/05/2022','13/08/2021'),('23/05/2021','12/09/2020')],
    'La Liga2':[('28/05/2023','12/08/2022'),('29/05/2022','13/08/2021'),('30/05/2021','12/09/2020')],
    'Bundesliga':[('27/05/2023','05/08/2022'),('14/05/2022','13/08/2021'),('22/05/2021','18/09/2020')],
    'Serie A':[('04/06/2023','13/08/2022'),('22/05/2022','21/08/2021'),('23/05/2021','19/09/2020')],
    'Serie B':[('19/05/2023','12/08/2022'),('06/05/2022','20/08/2021'),('10/05/2021','25/09/2020')],
    'Ligue 1':[('19/05/2023','05/08/2022'),('21/05/2022','06/08/2021'),('23/05/2021','21/08/2020')],
    'Ligue 2':[('02/06/2023','30/07/2022'),('14/05/2022','24/07/2021'),('15/05/2021','22/08/2020')],
    'Brasileirão Serie A':[('13/11/2022','09/04/2022'),('10/12/2021','29/05/2021'),('26/02/2021','08/08/2020')],
    'Primera División':[],
    'Major League Soccer':[('09/10/2022','26/02/2022'),('07/11/2021','17/04/2021'),('08/11/2020','29/02/2020')],
    'Eredivisie':[('28/05/2023','05/08/2022'),('15/05/2022','13/08/2021'),('16/05/2021','12/09/2020')],
    'Primeira Liga':[('27/05/2023','05/08/2022'),('15/05/2022','06/08/2021'),('19/05/2021','18/09/2020')],
    'J1 League':[('05/11/2022','18/02/2022'),('04/12/2021','26/02/2021'),('19/12/2020','21/02/2020')],
    'Scottish Premiership':[('23/04/2023','30/07/2022'),('10/04/2022','31/07/2021'),('21/02/2021','01/08/2020')],
    'Superliga':[('19/03/2023','15/07/2022'),('20/03/2022','16/07/2021'),('21/03/2021','11/09/2020')],
    'Süper Lig':[('07/06/2023','05/08/2022'),('22/05/2022','13/08/2021'),('15/05/2021','11/09/2020')],
    'Allsvenskan':[('06/11/2022','02/04/2022'),('04/12/2021','10/04/2021'),('06/12/2020','14/06/2020')],
    'Saudi Professional League':[('31/05/2023','25/08/2022'),('27/06/2022','11/08/2021'),('30/05/2021','17/10/2020')],
    'Jupiler Pro League':[('23/04/2023','22/07/2022'),('10/04/2022','23/07/2021'),('18/04/2021','08/08/2020')],
    'UEFA Champions League':[('10/06/2023','07/09/2022'),('28/05/2022','15/09/2021'),('29/05/2021','21/10/2020')],
    'UEFA Europa League':[('31/05/2023','08/09/2022'),('18/05/2022','16/09/2021'),('26/05/2021','22/10/2020')],
    'UEFA Europa Conference League':[('07/06/2023','08/09/2022'),('25/05/2022','14/09/2021'),()]
    }
#leagues

In [9]:
leagues_list = {
    'English Premier League':['https://www.flashscore.nl/voetbal/engeland/premier-league/schema/', 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/', 'https://www.sofascore.com/tournament/football/england/premier-league/17#52186'],
    'Championship':['https://www.flashscore.nl/voetbal/engeland/championship/schema/','https://www.worldfootball.net/all_matches/eng-championship-2023-2024/', 'https://www.sofascore.com/tournament/football/england/championship/18#52367'],
    'EFL Championship':['https://www.flashscore.nl/voetbal/engeland/efl-cup/schema/', '', 'https://www.sofascore.com/tournament/football/england/efl-cup/21#52393'],
    'EFL Trophy':['https://www.flashscore.nl/voetbal/engeland/efl-trophy/schema/', 'https://www.worldfootball.net/all_matches/eng-efl-trophy-2023-2024/', ''],
    'La Liga':['https://www.flashscore.nl/voetbal/spanje/laliga/schema/','https://www.worldfootball.net/all_matches/esp-primera-division-2023-2024/', 'https://www.sofascore.com/tournament/football/spain/laliga/8#52376'],
    'La Liga2':['https://www.flashscore.nl/voetbal/spanje/laliga2/schema/', 'https://www.worldfootball.net/all_matches/esp-segunda-division-2023-2024/', 'https://www.sofascore.com/tournament/football/spain/laliga-2/54#52563'],
    'Bundesliga':['https://www.flashscore.nl/voetbal/duitsland/bundesliga/schema/', 'https://www.worldfootball.net/all_matches/bundesliga-2023-2024/', 'https://www.sofascore.com/tournament/football/germany/bundesliga/35#52608'],
    'Serie A':['https://www.flashscore.nl/voetbal/italie/serie-a/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-a-2023-2024/', 'https://www.sofascore.com/tournament/football/italy/serie-a/23#52760'],
    'Serie B':['https://www.flashscore.nl/voetbal/italie/serie-b/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-b-2023-2024/', 'https://www.sofascore.com/tournament/football/italy/serie-b/53#52947'],
    'Ligue 1':['https://www.flashscore.nl/voetbal/frankrijk/ligue-1/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-1-2023-2024/', 'https://www.sofascore.com/tournament/football/france/ligue-1/34#52571'],
    'Ligue 2':['https://www.flashscore.nl/voetbal/frankrijk/ligue-2/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-2-2023-2024/', 'https://www.sofascore.com/tournament/football/france/ligue-2/182#52572'],
    'Brasileirão Serie A':['https://www.flashscore.nl/voetbal/brazilie/braziliaanse-competitie/schema/', 'https://www.worldfootball.net/all_matches/bra-serie-a-2023/', 'https://www.sofascore.com/tournament/football/brazil/brasileiro-serie-a/325#48982'],
    'Primera División':['https://www.flashscore.nl/voetbal/argentinie/primera-d/schema/', '', ''],
    'Major League Soccer':['https://www.flashscore.nl/voetbal/usa/mls/schema/', 'https://www.worldfootball.net/all_matches/usa-major-league-soccer-2023/', 'https://www.sofascore.com/tournament/football/usa/mls/242#47955'],
    'Eredivisie':['https://www.flashscore.nl/eredivisie/schema/', 'https://www.worldfootball.net/all_matches/ned-eredivisie-2023-2024/', 'https://www.sofascore.com/tournament/football/netherlands/eredivisie/37#52554'],
    'Primeira Liga':['https://www.flashscore.nl/voetbal/portugal/liga-portugal/schema/', 'https://www.worldfootball.net/all_matches/por-primeira-liga-2023-2024/', 'https://www.sofascore.com/tournament/football/portugal/liga-portugal/238#52769'],
    'J1 League':['https://www.flashscore.nl/voetbal/japan/j1-league/schema/', 'https://www.worldfootball.net/all_matches/jpn-j1-league-2023/', 'https://www.sofascore.com/tournament/football/japan/j1-league/196#48055'],
    'Scottish Premiership':['https://www.flashscore.nl/voetbal/schotland/premiership/schema/', 'https://www.worldfootball.net/all_matches/sco-premiership-2023-2024/', 'https://www.sofascore.com/tournament/football/scotland/premiership/36#52588'],
    'Superliga':['https://www.flashscore.nl/voetbal/denemarken/superliga/schema/', 'https://www.worldfootball.net/all_matches/den-superligaen-2023-2024/', 'https://www.sofascore.com/tournament/football/denmark/superliga/39#52172'],
    'Süper Lig':['https://www.flashscore.nl/voetbal/turkije/super-lig/schema/', 'https://www.worldfootball.net/all_matches/tur-sueperlig-2023-2024/', 'https://www.sofascore.com/tournament/football/turkey/trendyol-super-lig/52#53190'],
    'Allsvenskan':['https://www.flashscore.nl/voetbal/zweden/allsvenskan/schema/', 'https://www.worldfootball.net/all_matches/swe-allsvenskan-2023/', 'https://www.sofascore.com/tournament/football/sweden/allsvenskan/40#47730'],
    'Saudi Professional League':['https://www.flashscore.nl/voetbal/saoedi-arabie/premier-league/schema/', 'https://www.worldfootball.net/all_matches/ksa-saudi-pro-league-2023-2024/', 'https://www.sofascore.com/tournament/football/saudi-arabia/saudi-professional-league/955#53241'],
    'Jupiler Pro League':['https://www.flashscore.nl/voetbal/belgie/jupiler-pro-league/schema/', 'https://www.worldfootball.net/all_matches/bel-eerste-klasse-a-2023-2024/', 'https://www.sofascore.com/tournament/football/belgium/pro-league/38#52383'],
    'UEFA Champions League':['https://www.flashscore.nl/voetbal/europa/champions-league/schema/', 'https://www.worldfootball.net/all_matches/champions-league-2023-2024/', 'https://www.sofascore.com/tournament/football/europe/uefa-champions-league/7#52162'],
    'UEFA Europa League':['https://www.flashscore.nl/voetbal/europa/europa-league/schema/', 'https://www.worldfootball.net/all_matches/europa-league-2023-2024/', 'https://www.sofascore.com/tournament/football/europe/uefa-europa-league/679#53654'],
    'UEFA Europa Conference League':['https://www.flashscore.nl/voetbal/europa/europa-conference-league/schema/', 'https://www.worldfootball.net/all_matches/europa-conference-league-2023-2024/', 'https://www.sofascore.com/tournament/football/europe/uefa-europa-conference-league/17015#52327'],
}


---

In [10]:
%%time
#Total accurate prediction (Overall)
temp_list = list(df_corr_pred['accurate_prediction'])
filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
perc = len(filt_list)/df_corr_pred.shape[0]
perc

CPU times: total: 15.6 ms
Wall time: 17.2 ms


0.29639238337657253

In [11]:
%%time
#Total accurate prediction (by season)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_accuracy = {}

for season in season_list:
    season_accur = pd.DataFrame(columns=list(df_corr_pred.columns))
    for key in list(seasons.keys()):
        if (seasons[key] != []):
            if (seasons[key][season] != ()):
                upper_lim = seasons[key][season][0]
                original_date = datetime.strptime(upper_lim, "%d/%m/%Y")
                upper_lim = original_date.strftime("%Y-%m-%d")

                lower_lim = seasons[key][season][1]
                original_date = datetime.strptime(lower_lim, "%d/%m/%Y")
                lower_lim = original_date.strftime("%Y-%m-%d")

                temp_df = df_corr_pred[df_corr_pred['league'] == key]
                temp_df = temp_df[(temp_df['date'] <= upper_lim) & (temp_df['date'] >= lower_lim)]
                #print(temp_df.shape[0])
                season_accur = pd.concat([season_accur, temp_df], axis=0)

    #season_accur
    temp_list = list(season_accur['accurate_prediction'])
    filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
    perc = len(filt_list)/df_corr_pred.shape[0]
    season_accuracy[season_dict[str(season)]] = perc
                    
#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
season_acc_dist_sorted = {k: v for k, v in sorted(season_accuracy.items(), key=lambda item: item[1], reverse=True)}        
season_acc_dist_sorted         

CPU times: total: 453 ms
Wall time: 472 ms


{'2022/2021 Season': 0.10627049782172401,
 '2023/2022 Season': 0.10489989720495374,
 '2021/2020 Season': 0.08370453766704194}

In [12]:
#season_accur[season_accur['league'] == 'English Premier League']

In [13]:
%%time
#Total accurate prediction (By League)
league_acc_dist = {}
for league in leagues:
    temp_df = df_corr_pred[df_corr_pred['league'] == league]
    temp_list = list(temp_df['accurate_prediction'])
    filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
    if temp_df.shape[0] == 0:
        league_acc_dist[league] = 0
    else:
        perc = len(filt_list)/temp_df.shape[0]
        league_acc_dist[league] = perc

#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
league_acc_dist_sorted = {k: v for k, v in sorted(league_acc_dist.items(), key=lambda item: item[1], reverse=True)}        
league_acc_dist_sorted

CPU times: total: 93.8 ms
Wall time: 99.7 ms


{'Brasileirão Serie A': 0.35495495495495494,
 'Championship': 0.337468982630273,
 'Serie A': 0.335149863760218,
 'Major League Soccer': 0.3146853146853147,
 'Scottish Premiership': 0.3060869565217391,
 'Superliga': 0.30578512396694213,
 'La Liga': 0.3016304347826087,
 'Ligue 1': 0.3016304347826087,
 'Jupiler Pro League': 0.3010262257696693,
 'English Premier League': 0.3003629764065336,
 'Allsvenskan': 0.2960812772133527,
 'Serie B': 0.29363636363636364,
 'Bundesliga': 0.2912400455062571,
 'J1 League': 0.2909280500521376,
 'Saudi Professional League': 0.2895927601809955,
 'UEFA Champions League': 0.28493150684931506,
 'Primeira Liga': 0.28346456692913385,
 'La Liga2': 0.2828507795100223,
 'Ligue 2': 0.27661510464058237,
 'Süper Lig': 0.2730696798493409,
 'UEFA Europa League': 0.25333333333333335,
 'Eredivisie': 0.23917995444191345,
 'UEFA Europa Conference League': 0.23308270676691728,
 'EFL Trophy': 0.13385826771653545,
 'EFL Championship': 0,
 'Primera División': 0}

In [14]:
%%time

#Total Accurate Prediction (by season and league)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_league_accuracy = {}

for season in season_list:
    season_accur = pd.DataFrame(columns=list(df_corr_pred.columns))
    for key in list(seasons.keys()):
        if (seasons[key] != []):
            if (seasons[key][season] != ()):
                upper_lim = seasons[key][season][0]
                original_date = datetime.strptime(upper_lim, "%d/%m/%Y")
                upper_lim = original_date.strftime("%Y-%m-%d")

                lower_lim = seasons[key][season][1]
                original_date = datetime.strptime(lower_lim, "%d/%m/%Y")
                lower_lim = original_date.strftime("%Y-%m-%d")

                temp_df = df_corr_pred[df_corr_pred['league'] == key]
                temp_df = temp_df[(temp_df['date'] <= upper_lim) & (temp_df['date'] >= lower_lim)]
                #print(temp_df.shape[0])
                season_accur = pd.concat([season_accur, temp_df], axis=0)

    league_acc_dist = {}
    for league in leagues:
        temp_df = season_accur[season_accur['league'] == league]
        temp_list = list(temp_df['accurate_prediction'])
        filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
        if temp_df.shape[0] == 0:
            league_acc_dist[league] = 0
        else:
            perc = len(filt_list)/temp_df.shape[0]
            league_acc_dist[league] = perc
    league_acc_dist = {k: v for k, v in sorted(league_acc_dist.items(), key=lambda item: item[1], reverse=True)}
    season_league_accuracy[season_dict[str(season)]] = league_acc_dist
                    
#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
top_5 = []
for key in list(season_league_accuracy.keys()):
    print(f"{key}\n{season_league_accuracy[key]}\n\n")
    top_5 = top_5 + list(season_league_accuracy[key].keys())[:5]

#Top 5 types of prediction    
top_5 = Counter(top_5)
top_5 = {k: v for k, v in sorted(top_5.items(), key=lambda item: item[1], reverse=True)}
top_5

2023/2022 Season
{'Brasileirão Serie A': 0.3894736842105263, 'Serie A': 0.3684210526315789, 'Championship': 0.3418181818181818, 'Bundesliga': 0.33003300330033003, 'Superliga': 0.3230769230769231, 'La Liga': 0.32189973614775724, 'Scottish Premiership': 0.3181818181818182, 'Ligue 1': 0.3171428571428571, 'Jupiler Pro League': 0.31561461794019935, 'La Liga2': 0.29912663755458513, 'Major League Soccer': 0.296218487394958, 'UEFA Champions League': 0.29310344827586204, 'English Premier League': 0.2910662824207493, 'Saudi Professional League': 0.2895927601809955, 'Ligue 2': 0.2853333333333333, 'J1 League': 0.2721311475409836, 'UEFA Europa League': 0.2608695652173913, 'Primeira Liga': 0.2581699346405229, 'Serie B': 0.2572944297082228, 'Süper Lig': 0.2508038585209003, 'Allsvenskan': 0.24472573839662448, 'UEFA Europa Conference League': 0.24444444444444444, 'Eredivisie': 0.2138157894736842, 'EFL Trophy': 0.1693548387096774, 'EFL Championship': 0, 'Primera División': 0}


2022/2021 Season
{'Scotti

{'Brasileirão Serie A': 3,
 'Serie A': 2,
 'Championship': 2,
 'Bundesliga': 1,
 'Superliga': 1,
 'Scottish Premiership': 1,
 'Allsvenskan': 1,
 'Major League Soccer': 1,
 'UEFA Champions League': 1,
 'Serie B': 1,
 'Ligue 1': 1}

---

In [15]:
%%time
#Count of pattern in accurate prediction (Overall)
cause_dict = {}
temp_list = list(df_corr_pred['accurate_prediction'])
filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]

for item in filt_list:
    preds = item[list(item.keys())[0]]
    for pred in preds:
        if 'filtered' in pred:
            continue
        else:
            if pred in list(cause_dict.keys()):
                cause_dict[pred].append(1)
            else:
                cause_dict[pred] = [1]

cause_dict_sorted = {k: v for k, v in sorted(cause_dict.items(), key=lambda item: item[1], reverse=True)}        
                
for key in list(cause_dict_sorted.keys()):
    print(f"{key}: {len(cause_dict_sorted[key])}")

Score prediction for away team using away historic match scores, for only matches where away team played away role: : 1245
Score prediction for home role using home historic match scores, regardless of role played by home team: : 1224
Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: : 1202
Score prediction for away role using away historic match scores, regardless of role played by away team: : 1191
Score prediction for home team using home historic match scores, for only matches where home team played home role: : 1186
Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: : 1162
Score prediction for away team using away historic match scores after skipping rows, for only matches where away team played away role: : 640
Score prediction for home team using home historic match scores after skipping rows, for only matches where home team played

In [16]:
%%time
#Count of pattern in accurate prediction (By League)
league_count_dist = {}
for league in leagues:
    cause_dict = {}
    temp_df = df_corr_pred[df_corr_pred['league'] == league]
    temp_list = list(temp_df['accurate_prediction'])
    filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
    
    for item in filt_list:
        preds = item[list(item.keys())[0]]
        for pred in preds:
            if 'filtered' in pred:
                continue
            else:
                if pred in list(cause_dict.keys()):
                    cause_dict[pred].append(1)
                else:
                    cause_dict[pred] = [1]

    for key in list(cause_dict.keys()):
        cause_dict[key] = len(cause_dict[key])
    cause_dict_sorted = {k: v for k, v in sorted(cause_dict.items(), key=lambda item: item[1], reverse=True)}
    league_count_dist[league] = cause_dict_sorted

top_5 = []
for league in list(league_count_dist.keys()):
    print(f"{league}\n{league_count_dist[league]}\n")
    top_5 = top_5 + list(league_count_dist[league].keys())[:5]
    
#Top 5 types of prediction
top_5 = Counter(top_5)
top_5 = {k: v for k, v in sorted(top_5.items(), key=lambda item: item[1], reverse=True)}
top_5

English Premier League
{'Score prediction for away role using away historic match scores, regardless of role played by away team: ': 66, 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: ': 64, 'Score prediction for away team using away historic match scores, for only matches where away team played away role: ': 59, 'Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: ': 58, 'Score prediction for home role using home historic match scores, regardless of role played by home team: ': 50, 'Score prediction for home team using home historic match scores, for only matches where home team played home role: ': 48, 'Score prediction for away team using away historic match scores after skipping rows, for only matches where away team played away role: ': 28, 'Score prediction for both roles using head-to-head historic match scores, regardless of rol

UEFA Europa Conference League
{'Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: ': 12, 'Score prediction for home role using home historic match scores, regardless of role played by home team: ': 12, 'Score prediction for away role using away historic match scores, regardless of role played by away team: ': 11, 'Score prediction for home team using home historic match scores, for only matches where home team played home role: ': 11, 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: ': 10, 'Score prediction for away team using away historic match scores, for only matches where away team played away role: ': 10, 'Score prediction for home team using home historic match scores after skipping rows, for only matches where home team played home role: ': 6, 'Score prediction for away team using away historic match scores after skipping rows, 

{'Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: ': 22,
 'Score prediction for away team using away historic match scores, for only matches where away team played away role: ': 21,
 'Score prediction for home role using home historic match scores, regardless of role played by home team: ': 20,
 'Score prediction for home team using home historic match scores, for only matches where home team played home role: ': 20,
 'Score prediction for away role using away historic match scores, regardless of role played by away team: ': 19,
 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: ': 17,
 'Score prediction for home team using home historic match scores after skipping rows, for only matches where home team played home role: ': 1}

In [17]:
%%time
#Count of pattern in accurate prediction (by season)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_count_dist = {}

for season in season_list:
    season_accur = pd.DataFrame(columns=list(df_corr_pred.columns))
    for key in list(seasons.keys()):
        if (seasons[key] != []):
            if (seasons[key][season] != ()):
                upper_lim = seasons[key][season][0]
                original_date = datetime.strptime(upper_lim, "%d/%m/%Y")
                upper_lim = original_date.strftime("%Y-%m-%d")

                lower_lim = seasons[key][season][1]
                original_date = datetime.strptime(lower_lim, "%d/%m/%Y")
                lower_lim = original_date.strftime("%Y-%m-%d")

                temp_df = df_corr_pred[df_corr_pred['league'] == key]
                temp_df = temp_df[(temp_df['date'] <= upper_lim) & (temp_df['date'] >= lower_lim)]
                #print(temp_df.shape[0])
                season_accur = pd.concat([season_accur, temp_df], axis=0)

    cause_dict = {}
    temp_list = list(season_accur['accurate_prediction'])
    filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]

    for item in filt_list:
        preds = item[list(item.keys())[0]]
        for pred in preds:
            if 'filtered' in pred:
                continue
            else:
                if pred in list(cause_dict.keys()):
                    cause_dict[pred].append(1)
                else:
                    cause_dict[pred] = [1]

    for key in list(cause_dict.keys()):
        cause_dict[key] = len(cause_dict[key])
    cause_dict_sorted = {k: v for k, v in sorted(cause_dict.items(), key=lambda item: item[1], reverse=True)} 
    season_count_dist[season_dict[str(season)]] = cause_dict_sorted
                    
#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
top_5 = []
for key in list(season_count_dist.keys()):
    print(f"{key}\n{season_count_dist[key]}\n\n")
    top_5 = top_5 + list(season_count_dist[key].keys())[:5]
    
#Top 5 types of prediction
top_5 = Counter(top_5)
top_5 = {k: v for k, v in sorted(top_5.items(), key=lambda item: item[1], reverse=True)}
top_5

2023/2022 Season
{'Score prediction for home team using home historic match scores, for only matches where home team played home role: ': 436, 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: ': 423, 'Score prediction for home role using home historic match scores, regardless of role played by home team: ': 422, 'Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: ': 408, 'Score prediction for away team using away historic match scores, for only matches where away team played away role: ': 396, 'Score prediction for away role using away historic match scores, regardless of role played by away team: ': 393, 'Score prediction for home team using home historic match scores after skipping rows, for only matches where home team played home role: ': 246, 'Score prediction for away team using away historic match scores after skipping rows, for o

{'Score prediction for home role using home historic match scores, regardless of role played by home team: ': 3,
 'Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: ': 3,
 'Score prediction for away team using away historic match scores, for only matches where away team played away role: ': 3,
 'Score prediction for home team using home historic match scores, for only matches where home team played home role: ': 2,
 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: ': 2,
 'Score prediction for away role using away historic match scores, regardless of role played by away team: ': 2}

---

In [18]:
%%time
#Teams count in accurate predictions

team_corr_pred = df_corr_pred.copy(deep=True)
pred_list = list(team_corr_pred['accurate_prediction'])

accur_list = []
for item in pred_list:
    entry = 0
    if item[list(item.keys())[0]] != []:
        entry = 1
    accur_list.append(entry)
    
team_corr_pred['accur_categ'] = accur_list
team_corr_pred_1 = team_corr_pred[team_corr_pred['accur_categ'] == 1] 
teams = list(team_corr_pred_1['hometeam']) + list(team_corr_pred_1['awayteam'])

team_count = Counter(teams)
team_count = {k: v for k, v in sorted(team_count.items(), key=lambda item: item[1], reverse=True)}
team_count

CPU times: total: 15.6 ms
Wall time: 29.4 ms


{'Queens Park Rangers': 60,
 'Corinthians SP': 54,
 'Real Madrid': 53,
 'Huddersfield Town': 53,
 'AC Milan': 52,
 'Cardiff City': 51,
 'Birmingham City': 50,
 'Juventus': 49,
 'Millwall FC': 49,
 'Inter': 49,
 'Middlesbrough FC': 49,
 'Real Sociedad': 48,
 'West Ham United': 48,
 'Fenerbahçe': 47,
 'Sampdoria': 47,
 'Bristol City': 47,
 'Sevilla FC': 46,
 'Eintracht Frankfurt': 46,
 'Reading FC': 46,
 'US Cremonese': 45,
 'Manchester United': 45,
 'Atalanta': 45,
 'ACF Fiorentina': 44,
 'SPAL 2013 Ferrara': 44,
 'Preston North End': 44,
 'West Bromwich Albion': 44,
 'Fluminense RJ': 44,
 'Atlético Madrid': 43,
 'RC Strasbourg': 43,
 'Stade Rennais': 43,
 'CD Lugo': 43,
 'Arsenal FC': 43,
 'Club Brugge KV': 43,
 'Venezia FC': 43,
 'Swansea City': 43,
 'SSC Napoli': 43,
 'Lazio Roma': 42,
 'Brighton & Hove Albion': 42,
 'Chelsea FC': 42,
 'Fulham FC': 42,
 'Stoke City': 42,
 'Atlético Mineiro': 42,
 'Olympique Lyon': 41,
 'ESTAC Troyes': 41,
 'Udinese Calcio': 41,
 'Girona FC': 41,
 'Co

In [19]:
list_of_teams = list(team_count.keys())
#list_of_teams

In [20]:
%%time

#Teams accuracy during occurence
team_accuracy = {}

for team in list_of_teams:
    temp_df = df_corr_pred[(df_corr_pred['hometeam'] == team) | (df_corr_pred['awayteam'] == team)]
    team_corr_pred = temp_df.copy(deep=True)
    
    temp_list = list(team_corr_pred['accurate_prediction'])
    filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
    perc = len(filt_list)/team_corr_pred.shape[0]
    team_accuracy[team] = perc

team_accuracy = {k: v for k, v in sorted(team_accuracy.items(), key=lambda item: item[1], reverse=True)}
team_accuracy

CPU times: total: 3.12 s
Wall time: 3.18 s


{'FK Krasnodar': 0.625,
 'HNK Rijeka': 0.5,
 'Apollon Limassol': 0.5,
 'NŠ Mura': 0.5,
 'FC Alashkert': 0.5,
 'Dundalk FC': 0.5,
 'Corinthians SP': 0.4864864864864865,
 'FC Nordsjælland': 0.45,
 'Queens Park Rangers': 0.4444444444444444,
 'Al Ettifaq': 0.4444444444444444,
 'Motherwell FC': 0.425531914893617,
 'Sampdoria': 0.42342342342342343,
 'Chapecoense': 0.42105263157894735,
 'Goiás - GO': 0.4189189189189189,
 'Cuiabá - MT': 0.41333333333333333,
 'New York RB': 0.4117647058823529,
 'Los Angeles FC': 0.40963855421686746,
 'Abha Club': 0.4074074074074074,
 'Al Tai': 0.4074074074074074,
 'US Cremonese': 0.40540540540540543,
 'Cercle Brugge': 0.40404040404040403,
 'Juventude - RS': 0.4,
 'Accrington Stanley': 0.4,
 'FK Žalgiris': 0.4,
 'FC Zürich': 0.4,
 'Spartak Moskva': 0.4,
 'SPAL 2013 Ferrara': 0.3963963963963964,
 'Fluminense RJ': 0.3963963963963964,
 'América - MG': 0.39473684210526316,
 'Avaí - SC': 0.39473684210526316,
 'Huddersfield Town': 0.3925925925925926,
 'AD Alcorcón': 0

In [21]:
%%time

#Total teams count in accurate predictions (by season)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_accuracy = {}

for season in season_list:
    season_accur = pd.DataFrame(columns=list(df_corr_pred.columns))
    for key in list(seasons.keys()):
        if (seasons[key] != []):
            if (seasons[key][season] != ()):
                upper_lim = seasons[key][season][0]
                original_date = datetime.strptime(upper_lim, "%d/%m/%Y")
                upper_lim = original_date.strftime("%Y-%m-%d")

                lower_lim = seasons[key][season][1]
                original_date = datetime.strptime(lower_lim, "%d/%m/%Y")
                lower_lim = original_date.strftime("%Y-%m-%d")

                temp_df = df_corr_pred[df_corr_pred['league'] == key]
                temp_df = temp_df[(temp_df['date'] <= upper_lim) & (temp_df['date'] >= lower_lim)]
                #print(temp_df.shape[0])
                season_accur = pd.concat([season_accur, temp_df], axis=0)
    
    #Teams count in given season
    team_accuracy = {}

    team_corr_pred = season_accur.copy(deep=True)
    pred_list = list(team_corr_pred['accurate_prediction'])

    accur_list = []
    for item in pred_list:
        entry = 0
        if item[list(item.keys())[0]] != []:
            entry = 1
        accur_list.append(entry)

    team_corr_pred['accur_categ'] = accur_list
    team_corr_pred_1 = team_corr_pred[team_corr_pred['accur_categ'] == 1] 
    teams = list(team_corr_pred_1['hometeam']) + list(team_corr_pred_1['awayteam'])

    team_count = Counter(teams)
    team_count = {k: v for k, v in sorted(team_count.items(), key=lambda item: item[1], reverse=True)}
    season_accuracy[season_dict[str(season)]] = team_count
    
#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
top_5 = []
for key in list(season_accuracy.keys()):
    print(f"{key}\n{season_accuracy[key]}\n\n")
    top_5 = top_5 + list(season_accuracy[key].keys())[:5]
    
#Top 5 types of prediction
top_5 = Counter(top_5)
top_5 = {k: v for k, v in sorted(top_5.items(), key=lambda item: item[1], reverse=True)}
top_5

2023/2022 Season
{'Corinthians SP': 24, 'Huddersfield Town': 22, 'Juventus': 22, 'Rotherham United': 21, 'ACF Fiorentina': 21, 'Inter': 21, 'Manchester United': 20, 'Birmingham City': 20, 'Cardiff City': 20, 'Reading FC': 20, 'Goiás - GO': 20, 'Queens Park Rangers': 19, 'Bayer Leverkusen': 19, 'Eintracht Frankfurt': 19, 'US Cremonese': 19, 'West Ham United': 18, 'Chelsea FC': 18, 'Bristol City': 18, 'West Bromwich Albion': 18, 'Real Sociedad': 18, 'Real Madrid': 18, 'Sampdoria': 18, 'US Lecce': 18, 'AC Milan': 18, 'Ceará - CE': 18, 'Atlético Mineiro': 18, 'KV Oostende': 18, 'Everton FC': 17, 'Coventry City': 17, 'Luton Town': 17, 'Norwich City': 17, 'Lazio Roma': 17, 'SPAL 2013 Ferrara': 17, 'RC Strasbourg': 17, 'América - MG': 17, 'Atlético Goianiense': 17, 'Sunderland AFC': 16, 'Getafe CF': 16, 'FC Cartagena': 16, 'Bologna FC': 16, 'Udinese Calcio': 16, 'Reggina 1914': 16, 'Stade Rennais': 16, 'Preston North End': 15, 'Blackburn Rovers': 15, 'Real Betis': 15, 'Atlético Madrid': 15, '

{'Queens Park Rangers': 2,
 'Corinthians SP': 1,
 'Huddersfield Town': 1,
 'Juventus': 1,
 'Rotherham United': 1,
 'ACF Fiorentina': 1,
 'Fenerbahçe': 1,
 'West Ham United': 1,
 'Middlesbrough FC': 1,
 'AFC Bournemouth': 1,
 'Barnsley FC': 1,
 'FC Nantes': 1,
 'Paris Saint-Germain': 1,
 'Lille OSC': 1}

In [22]:
%%time

#Total team accuracy during occurence (by season)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_accuracy = {}

for season in season_list:
    season_accur = pd.DataFrame(columns=list(df_corr_pred.columns))
    for key in list(seasons.keys()):
        if (seasons[key] != []):
            if (seasons[key][season] != ()):
                upper_lim = seasons[key][season][0]
                original_date = datetime.strptime(upper_lim, "%d/%m/%Y")
                upper_lim = original_date.strftime("%Y-%m-%d")

                lower_lim = seasons[key][season][1]
                original_date = datetime.strptime(lower_lim, "%d/%m/%Y")
                lower_lim = original_date.strftime("%Y-%m-%d")

                temp_df = df_corr_pred[df_corr_pred['league'] == key]
                temp_df = temp_df[(temp_df['date'] <= upper_lim) & (temp_df['date'] >= lower_lim)]
                #print(temp_df.shape[0])
                season_accur = pd.concat([season_accur, temp_df], axis=0)
    
    #Teams accuracy during occurence in given season
    team_accuracy = {}

    for team in list_of_teams:
        temp_df = season_accur[(season_accur['hometeam'] == team) | (season_accur['awayteam'] == team)]
        team_corr_pred = temp_df.copy(deep=True)

        temp_list = list(team_corr_pred['accurate_prediction'])
        filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
        if team_corr_pred.shape[0] == 0:
            team_accuracy[team] = 0
        else:
            perc = len(filt_list)/team_corr_pred.shape[0]
            team_accuracy[team] = perc
        
    team_accuracy = {k: v for k, v in sorted(team_accuracy.items(), key=lambda item: item[1], reverse=True)}
    season_accuracy[season_dict[str(season)]] = team_accuracy
    
#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
top_5 = []
for key in list(season_accuracy.keys()):
    print(f"{key}\n{season_accuracy[key]}\n\n")
    top_5 = top_5 + list(season_accuracy[key].keys())[:5]
    
#Top 5 types of prediction
top_5 = Counter(top_5)
top_5 = {k: v for k, v in sorted(top_5.items(), key=lambda item: item[1], reverse=True)}
top_5

2023/2022 Season
{'Cambridge United': 0.6666666666666666, 'Corinthians SP': 0.631578947368421, 'KV Oostende': 0.5294117647058824, 'Goiás - GO': 0.5263157894736842, 'US Cremonese': 0.5, 'Viborg FF': 0.5, 'Crvena Zvezda': 0.5, 'CFR Cluj': 0.5, 'HJK Helsinki': 0.5, 'Apollon Limassol': 0.5, 'Newport County': 0.5, 'Arsenal FC U23': 0.5, 'Wolverhampton Wanderers U23': 0.5, 'Aston Villa U23': 0.5, 'RC Strasbourg': 0.4857142857142857, 'Everton FC': 0.4857142857142857, 'Huddersfield Town': 0.4782608695652174, 'Sampdoria': 0.47368421052631576, 'Atlético Mineiro': 0.47368421052631576, 'Ceará - CE': 0.47368421052631576, 'US Lecce': 0.47368421052631576, 'Silkeborg IF': 0.4642857142857143, 'Rotherham United': 0.45652173913043476, 'FC Nordsjælland': 0.45454545454545453, 'Eintracht Frankfurt': 0.4523809523809524, 'SPAL 2013 Ferrara': 0.4473684210526316, 'Atlético Goianiense': 0.4473684210526316, 'América - MG': 0.4473684210526316, 'Al Ettifaq': 0.4444444444444444, 'Cercle Brugge': 0.4411764705882353, 

{'Cambridge United': 1,
 'Corinthians SP': 1,
 'KV Oostende': 1,
 'Goiás - GO': 1,
 'US Cremonese': 1,
 'Cheltenham Town': 1,
 'Bristol Rovers': 1,
 'Olympiakos Piraeus': 1,
 'Dinamo Kiev': 1,
 'Motherwell FC': 1,
 'Leyton Orient': 1,
 'FK Krasnodar': 1,
 'Inter Miami CF': 1,
 'IFK Göteborg': 1,
 'FC Tokyo': 1}

---

In [23]:
%%time

##Weekly trend (by season and league)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_accuracy = {}

for season in season_list:
    season_accur = pd.DataFrame(columns=list(df_corr_pred.columns))
    leagues_trend = {}
    for key in list(seasons.keys()):
        league_trend = []
        if (seasons[key] != []):
            if (seasons[key][season] != ()):
                upper_lim = seasons[key][season][0]
                original_date_up = datetime.strptime(upper_lim, "%d/%m/%Y")
                upper_lim = original_date_up.strftime("%Y-%m-%d")

                lower_lim = seasons[key][season][1]
                original_date_lo = datetime.strptime(lower_lim, "%d/%m/%Y")
                lower_lim = original_date_lo.strftime("%Y-%m-%d")

                temp_df = df_corr_pred[df_corr_pred['league'] == key]
                temp_df = temp_df[(temp_df['date'] <= upper_lim) & (temp_df['date'] >= lower_lim)]
                
                current_lowerlim = original_date_lo
                next_weeklim = current_lowerlim + timedelta(days=7)
                
                while next_weeklim <= original_date_up:
                    temp_df_1 = temp_df[(temp_df['date'] <= next_weeklim) & (temp_df['date'] >= current_lowerlim)]
                    current_lowerlim = next_weeklim
                    next_weeklim = current_lowerlim + timedelta(days=7)
                    
                    temp_list = list(temp_df_1['accurate_prediction'])
                    filt_list = [item for item in temp_list if item[list(item.keys())[0]] != []]
                    if temp_df_1.shape[0] == 0:
                        league_trend.append(0)
                    else:
                        perc = len(filt_list)/temp_df_1.shape[0]
                        league_trend.append(perc)
                        
        leagues_trend[key] = league_trend
    season_accuracy[season_dict[str(season)]] = leagues_trend
    
for key in list(season_accuracy.keys()):
    print(f"{key}\n{season_accuracy[key]}\n\n")

2023/2022 Season
{'English Premier League': [0.5, 0.1, 0.2, 0.15, 0.4, 0.0, 0.0, 0, 0.3, 0.45454545454545453, 0.2631578947368421, 0.4, 0.2, 0.3, 0.3, 0, 0, 0, 0, 0, 0.4166666666666667, 0.4, 0.5, 0.16666666666666666, 0.3, 0.0, 0.36363636363636365, 0.36363636363636365, 0.0, 0.1, 0.3, 0.3076923076923077, 0.14285714285714285, 0, 0.1875, 0.2, 0.5454545454545454, 0.3888888888888889, 0.3076923076923077], 'Championship': [0.25, 0.2727272727272727, 0.20833333333333334, 0.4166666666666667, 0.4, 0.25, 0.3333333333333333, 0.25, 0.0, 0.36, 0.2857142857142857, 0.3333333333333333, 0.5333333333333333, 0.52, 0.3333333333333333, 0.5, 0, 0, 0.0, 0.5384615384615384, 0.5833333333333334, 0.4166666666666667, 0.3125, 0, 0.35714285714285715, 0.36363636363636365, 0.4, 0.2857142857142857, 0.24, 0.11764705882352941, 0.4, 0.4375, 0.3333333333333333, 0.3333333333333333, 0.0, 0.20833333333333334, 0.2, 0.3333333333333333, 0.3333333333333333, 0.46153846153846156], 'EFL Championship': [], 'EFL Trophy': [0.1935483870967

In [ ]:
'Brasileirão Serie A': 0.46153846153846156, 0.26666666666666666, 0.3076923076923077, 0.4, 0.26666666666666666, 0.5, 0.36363636363636365, 0.26666666666666666, 0.5, 0.42857142857142855, 0.5, 0.5, 0.5833333333333334, 0.23076923076923078, 0.3333333333333333, 0.3076923076923077, 0.42857142857142855, 0.4166666666666667, 0.25, 0.46153846153846156, 0.5384615384615384, 0.35714285714285715, 0.2727272727272727, 0.4, 0.29411764705882354, 0.55, 0.2, 0.23076923076923078, 0.5238095238095238, 0.4375, 0.3333333333333333
'Brasileirão Serie A': 0.1, 0.18181818181818182, 0.2222222222222222, 0.6842105263157895, 0.3181818181818182, 0.3076923076923077, 0.15384615384615385, 0.3076923076923077, 0.18181818181818182, 0.2727272727272727, 0.18181818181818182, 0.38461538461538464, 0.16666666666666666, 0.6363636363636364, 0.25, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384, 0.5652173913043478, 0.23809523809523808, 0.375, 0.1875, 0.4375, 0.391304347826087, 0.4090909090909091, 0.4117647058823529, 0.3125
'Brasileirão Serie A': 0, 0.0, 0.15384615384615385, 0.35, 0.4090909090909091, 0.3076923076923077, 0.5, 0.38461538461538464, 0.391304347826087, 0.45454545454545453, 0.2857142857142857, 0.5, 0.25, 0.35294117647058826, 0.4, 0.5, 0.46153846153846156, 0.21428571428571427, 0.15384615384615385, 0.18181818181818182, 0.5, 0.2, 0.3333333333333333, 0.23809523809523808, 0.21428571428571427, 0.21052631578947367, 0.3333333333333333, 0.5   

'Superliga': 0.14285714285714285, 0.5, 0.375, 0.125, 0.16666666666666666, 0.3333333333333333, 0.5714285714285714, 0.3333333333333333, 0.2857142857142857, 0.3333333333333333, 0.0, 0.5714285714285714, 0.42857142857142855, 0.42857142857142855, 0.42857142857142855, 0.2857142857142857, 0.5, 0.16666666666666666, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.42857142857142855, 0.5714285714285714, 0.3333333333333333
'Superliga': 0.0, 0.0, 0.0, 0.125, 0.0, 0.5, 0.3333333333333333, 0.0, 0.2857142857142857, 0.16666666666666666, 0.14285714285714285, 0.3333333333333333, 0.0, 0.5714285714285714, 0.5, 0.5714285714285714, 0.16666666666666666, 0.0, 0.14285714285714285, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.2857142857142857, 0.25, 0.2857142857142857, 0.3333333333333333    
'Superliga': 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    
'Primeira Liga': 0.1, 0.0, 0.2222222222222222, 0.16666666666666666, 0.4, 0.3, 0.1111111111111111, 0.0, 0.5, 0.3333333333333333, 1.0, 0.2, 0.3, 0.2222222222222222, 0.1111111111111111, 0, 0, 0, 0, 0.0, 0.125, 0.16666666666666666, 0.2222222222222222, 0.18181818181818182, 0.4, 0.1111111111111111, 0.2727272727272727, 0.3333333333333333, 0.3, 0.18181818181818182, 0.2, 0.5, 0.0, 0.0, 0.3076923076923077, 0.4, 0.2, 0.3, 0.3, 0.4, 0.5, 0.18181818181818182
'Primeira Liga': 0.2222222222222222, 0.0, 0.18181818181818182, 0.4444444444444444, 0, 0.3, 0.2727272727272727, 0.6363636363636364, 0.3333333333333333, 0, 0.0, 0.4, 0.2727272727272727, 0.0, 0, 1.0, 0.2727272727272727, 0.25, 0.09090909090909091, 0.2222222222222222, 0.125, 0.3333333333333333, 0.3, 0.6, 0.5, 0.4444444444444444, 0.3, 0.1, 0.2, 0.3, 0.5, 0.3, 0.2222222222222222, 1.0, 0.3, 0.16666666666666666, 0.3, 0.2, 0.3, 0.5
'Primeira Liga': 0, 0, 0, 0, 0.1111111111111111, 0.2727272727272727, 0.1, 0.25, 0, 0.5, 0.4, 0.4444444444444444, 1.0, 0.6666666666666666, 0.2222222222222222, 0.5384615384615384, 0.5, 0.3333333333333333, 0.2222222222222222, 0.2222222222222222, 0.5333333333333333, 0.5454545454545454, 0.3, 0.2727272727272727, 0.3, 0.4, 0.0, 0.0, 0.3, 0.09090909090909091, 0.2777777777777778, 0.6153846153846154, 0.3125, 0.23076923076923078

In [27]:
from scipy.spatial.distance import euclidean
import numpy as np

# Brasileirão Serie A time series data
time_series1 = np.array([0.46153846153846156, 0.26666666666666666, 0.3076923076923077, 0.4, 0.26666666666666666, 0.5, 0.36363636363636365, 0.26666666666666666, 0.5, 0.42857142857142855, 0.5, 0.5, 0.5833333333333334, 0.23076923076923078, 0.3333333333333333, 0.3076923076923077, 0.42857142857142855, 0.4166666666666667, 0.25, 0.46153846153846156, 0.5384615384615384, 0.35714285714285715, 0.2727272727272727, 0.4, 0.29411764705882354, 0.55, 0.2])
time_series2 = np.array([0.1, 0.18181818181818182, 0.2222222222222222, 0.6842105263157895, 0.3181818181818182, 0.3076923076923077, 0.15384615384615385, 0.3076923076923077, 0.18181818181818182, 0.2727272727272727, 0.18181818181818182, 0.38461538461538464, 0.16666666666666666, 0.6363636363636364, 0.25, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384, 0.5652173913043478, 0.23809523809523808, 0.375, 0.1875, 0.4375, 0.391304347826087, 0.4090909090909091, 0.4117647058823529, 0.3125])
time_series3 = np.array([0, 0.0, 0.15384615384615385, 0.35, 0.4090909090909091, 0.3076923076923077, 0.5, 0.38461538461538464, 0.391304347826087, 0.45454545454545453, 0.2857142857142857, 0.5, 0.25, 0.35294117647058826, 0.4, 0.5, 0.46153846153846156, 0.21428571428571427, 0.15384615384615385, 0.18181818181818182, 0.5, 0.2, 0.3333333333333333, 0.23809523809523808, 0.21428571428571427, 0.21052631578947367, 0.3333333333333333])


#Calculate Euclidean similarity
#2023/2022 - 2022/2021
euclidean_distance = euclidean(time_series1, time_series2)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2023/2022 - 2022/2021')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

#2023/2022 - 2021/2020
euclidean_distance = euclidean(time_series1, time_series3)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2023/2022 - 2021/2020')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

#2022/2021 - 2021/2020
euclidean_distance = euclidean(time_series2, time_series3)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2022/2021 - 2021/2020')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

2023/2022 - 2022/2021
Euclidean Distance: 1.0936837769573313
Euclidean Similarity: 0.4776270471242132
2023/2022 - 2021/2020
Euclidean Distance: 0.9751561365526368
Euclidean Similarity: 0.5062890884896636
2022/2021 - 2021/2020
Euclidean Distance: 0.9945489745285581
Euclidean Similarity: 0.5013664807284891


In [29]:
# Superliga time series data
time_series1 = np.array([0.14285714285714285, 0.5, 0.375, 0.125, 0.16666666666666666, 0.3333333333333333, 0.5714285714285714, 0.3333333333333333, 0.2857142857142857, 0.3333333333333333, 0.0, 0.5714285714285714, 0.42857142857142855, 0.42857142857142855, 0.42857142857142855, 0.2857142857142857, 0.5, 0.16666666666666666, 0.0, 0.0])
time_series2 = np.array([0.0, 0.0, 0.0, 0.125, 0.0, 0.5, 0.3333333333333333, 0.0, 0.2857142857142857, 0.16666666666666666, 0.14285714285714285, 0.3333333333333333, 0.0, 0.5714285714285714, 0.5, 0.5714285714285714, 0.16666666666666666, 0.0, 0.14285714285714285, 0.3333333333333333])

#Calculate Euclidean similarity
#2023/2022 - 2022/2021
euclidean_distance = euclidean(time_series1, time_series2)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2023/2022 - 2022/2021')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

2023/2022 - 2022/2021
Euclidean Distance: 1.1403898217619408
Euclidean Similarity: 0.4672046137730243


In [30]:
# Primeira Liga time series data
time_series1 = np.array([0.1, 0.0, 0.2222222222222222, 0.16666666666666666, 0.4, 0.3, 0.1111111111111111, 0.0, 0.5, 0.3333333333333333, 1.0, 0.2, 0.3, 0.2222222222222222, 0.1111111111111111, 0, 0, 0, 0, 0.0, 0.125, 0.16666666666666666, 0.2222222222222222, 0.18181818181818182, 0.4, 0.1111111111111111, 0.2727272727272727])
time_series2 = np.array([0.2222222222222222, 0.0, 0.18181818181818182, 0.4444444444444444, 0, 0.3, 0.2727272727272727, 0.6363636363636364, 0.3333333333333333, 0, 0.0, 0.4, 0.2727272727272727, 0.0, 0, 1.0, 0.2727272727272727, 0.25, 0.09090909090909091, 0.2222222222222222, 0.125, 0.3333333333333333, 0.3, 0.6, 0.5, 0.4444444444444444, 0.3])
time_series3 = np.array([0, 0, 0, 0, 0.1111111111111111, 0.2727272727272727, 0.1, 0.25, 0, 0.5, 0.4, 0.4444444444444444, 1.0, 0.6666666666666666, 0.2222222222222222, 0.5384615384615384, 0.5, 0.3333333333333333, 0.2222222222222222, 0.2222222222222222, 0.5333333333333333, 0.5454545454545454, 0.3, 0.2727272727272727, 0.3, 0.4, 0.0])


#Calculate Euclidean similarity
#2023/2022 - 2022/2021
euclidean_distance = euclidean(time_series1, time_series2)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2023/2022 - 2022/2021')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

#2023/2022 - 2021/2020
euclidean_distance = euclidean(time_series1, time_series3)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2023/2022 - 2021/2020')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

#2022/2021 - 2021/2020
euclidean_distance = euclidean(time_series2, time_series3)
euclidean_similarity = 1 / (1 + euclidean_distance)

# Display the results
print('2022/2021 - 2021/2020')
print("Euclidean Distance:", euclidean_distance)
print("Euclidean Similarity:", euclidean_similarity)

2023/2022 - 2022/2021
Euclidean Distance: 1.8577557022822753
Euclidean Similarity: 0.34992494256992474
2023/2022 - 2021/2020
Euclidean Distance: 1.6951626737808072
Euclidean Similarity: 0.37103511774196096
2022/2021 - 2021/2020
Euclidean Distance: 1.6606276451461792
Euclidean Similarity: 0.3758511649776752


In [24]:
%%time

#Weekly trend gradient (by season and league)
season_dict = {'0':'2023/2022 Season','1':'2022/2021 Season','2':'2021/2020 Season'}
season_list = [0,1,2]
season_grad = {}
for key in list(season_accuracy.keys()):
    leagues_grad = {}
    for league in list(season_accuracy[key].keys()):
        if len(season_accuracy[key][league]) > 0:
            grad = (season_accuracy[key][league][-1] - season_accuracy[key][league][0])/len(season_accuracy[key][league])
            leagues_grad[league] = grad
        else:
            leagues_grad[league] = 0
    
    leagues_grad = {k: v for k, v in sorted(leagues_grad.items(), key=lambda item: item[1], reverse=True)}
    season_grad[key] = leagues_grad
    
#To sort the dictionary by descending order (to change to ascending order, remove the 'reverse' argument.)
top_5 = []
for key in list(season_grad.keys()):
    print(f"{key}\n{season_grad[key]}\n\n")
    top_5 = top_5 + list(season_grad[key].keys())[:5]
    
#Top 5 types of prediction
top_5 = Counter(top_5)
top_5 = {k: v for k, v in sorted(top_5.items(), key=lambda item: item[1], reverse=True)}
top_5

2023/2022 Season
{'Saudi Professional League': 0.011655011655011654, 'Serie A': 0.009549973835688123, 'Süper Lig': 0.0066445182724252485, 'Bundesliga': 0.006547619047619048, 'Superliga': 0.005442176870748299, 'Championship': 0.005288461538461539, 'La Liga2': 0.0040650406504065045, 'La Liga': 0.003968253968253969, 'Serie B': 0.003749999999999999, 'Scottish Premiership': 0.0026315789473684206, 'Primeira Liga': 0.001948051948051948, 'Jupiler Pro League': 0.0019230769230769234, 'EFL Championship': 0, 'Ligue 1': 0.0, 'Primera División': 0, 'Allsvenskan': 0.0, 'J1 League': -0.0013650013650013649, 'UEFA Europa Conference League': -0.0027223230490018148, 'Ligue 2': -0.003671970624235006, 'Brasileirão Serie A': -0.004135649296939621, 'Major League Soccer': -0.004140625, 'English Premier League': -0.004930966469428007, 'UEFA Champions League': -0.005574136008918617, 'UEFA Europa League': -0.006306306306306306, 'EFL Trophy': -0.0064516129032258064, 'Eredivisie': -0.011904761904761904}


2022/2021

{'Superliga': 2,
 'Brasileirão Serie A': 2,
 'Primeira Liga': 2,
 'Saudi Professional League': 1,
 'Serie A': 1,
 'Süper Lig': 1,
 'Bundesliga': 1,
 'Eredivisie': 1,
 'J1 League': 1,
 'Serie B': 1,
 'Ligue 2': 1,
 'English Premier League': 1}